In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import metrics
from collections import Counter
%matplotlib inline 
import seaborn as sns
sns.set_style('ticks')
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, roc_auc_score
from sklearn import linear_model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import statsmodels.api as sm
from scipy import stats
from sklearn.metrics import confusion_matrix
import xgboost as xgb
from xgboost import plot_importance
import warnings
from datetime import datetime, timedelta
from fbprophet import Prophet
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

Importing plotly failed. Interactive plots will not work.


In [2]:
df = pd.read_csv('file:///Users/chrisfiorentine/Downloads/game_logs.csv')

In [3]:
df = df[df['date'] > 20150000]

In [4]:
#df['date'] = df['date'].apply(lambda date: str(date))

In [5]:
#df['date'] = df['date'].apply(lambda date: datetime(year=int(date[0:4]), month=int(date[4:6]), 
                                                   # day=int(date[6:8])))

In [6]:
df['Run_dif'] = df.h_score - df.v_score

In [7]:
def wins(rd):
    if rd > 0:
        return 1
    if rd < 0:
        return 0

In [8]:
df['Win'] = df['Run_dif'].apply(wins)

In [9]:
drop_columns = ['v_line_score','h_line_score','1b_umpire_id', '1b_umpire_name','2b_umpire_id','2b_umpire_name',
                   '3b_umpire_id','3b_umpire_name','lf_umpire_id','lf_umpire_name','rf_umpire_id','rf_umpire_name',
                   'winning_rbi_batter_id','winning_rbi_batter_id_name','winning_pitcher_id',
                   'winning_pitcher_name','losing_pitcher_id','losing_pitcher_name','saving_pitcher_id',
                   'saving_pitcher_name','additional_info','acquisition_info','number_of_game','day_of_week',
                    'v_league','v_game_number','h_league','h_game_number','length_outs','day_night',
                   'completion','forefeit','protest','park_id','length_minutes','v_starting_pitcher_name','v_player_1_id','v_player_1_name',
                     'v_player_1_def_pos','v_player_2_id','v_player_3_id','v_player_4_id','v_player_5_id',
                     'v_player_6_id','v_player_7_id','v_player_8_id','v_player_9_id','v_player_2_name',
                     'v_player_3_name','v_player_4_name','v_player_5_name','v_player_6_name','v_player_7_name',
                     'v_player_8_name','v_player_9_name','v_player_2_def_pos','v_player_3_def_pos',
                     'v_player_4_def_pos','v_player_5_def_pos','v_player_6_def_pos','v_player_7_def_pos',
                     'v_player_8_def_pos','v_player_9_def_pos','h_player_2_def_pos', 'h_player_3_id', 'h_player_3_name',
       'h_player_3_def_pos', 'h_player_4_id', 'h_player_4_name',
       'h_player_4_def_pos', 'h_player_5_id', 'h_player_5_name',
       'h_player_5_def_pos', 'h_player_6_id', 'h_player_6_name',
       'h_player_6_def_pos', 'h_player_7_id', 'h_player_7_name',
       'h_player_7_def_pos', 'h_player_8_id', 'h_player_8_name',
       'h_player_8_def_pos', 'h_player_9_id', 'h_player_9_name',
       'h_player_9_def_pos','hp_umpire_id', 'hp_umpire_name',
       'h_manager_id', 'h_manager_name',
       'h_starting_pitcher_name', 'h_player_1_id', 'h_player_1_name','v_manager_id', 'v_manager_name',
       'h_player_1_def_pos', 'h_player_2_id', 'h_player_2_name','v_starting_pitcher_id','h_starting_pitcher_id']

In [10]:
df = df.drop(columns = drop_columns)

In [11]:
df.dropna(inplace = True)

In [12]:
df['h_xbh'] = df.h_doubles + df.h_triples + df.h_homeruns

In [13]:
df = df.drop(columns = ['h_doubles','h_triples','h_homeruns'])

In [14]:
df['v_xbh'] = df.v_doubles + df.v_triples + df.v_homeruns

In [15]:
df = df.drop(columns = ['v_doubles','v_triples','v_homeruns'])

In [16]:
df['v_pitcher_allowed_on_base'] = df.h_walks + df['h_intentional walks'] + df.h_hit_by_pitch

In [17]:
df['h_pitcher_allowed_on_base'] = df.v_walks + df['v_intentional walks'] + df.v_hit_by_pitch

In [18]:
df = df.drop(columns = ['v_walks','v_intentional walks','h_hit_by_pitch'])

In [19]:
df = df.drop(columns = ['h_walks','h_intentional walks','v_hit_by_pitch'])

In [20]:
df['v_advanced_runner'] = df.v_sacrifice_hits + df.v_sacrifice_flies +df.v_stolen_bases

In [21]:
df['h_advanced_runner'] = df.h_sacrifice_hits + df.h_sacrifice_flies +df.h_stolen_bases

In [22]:
df = df.drop(columns = ['h_sacrifice_hits','h_sacrifice_flies','h_stolen_bases'])

In [23]:
df = df.drop(columns = ['v_sacrifice_hits','v_sacrifice_flies','v_stolen_bases'])

In [24]:
df['v_pitcher_allowed_runner_advance'] = df.v_wild_pitches + df.v_balks + df.v_passed_balls

In [25]:
df['h_pitcher_allowed_runner_advance'] = df.h_wild_pitches + df.h_balks + df.h_passed_balls

In [26]:
df = df.drop(columns = ['v_wild_pitches','v_balks','v_passed_balls','h_wild_pitches','h_balks','h_passed_balls'])

In [27]:
train = df.head(3340)
start_of_season_df = train.tail(913)

In [28]:
home_start_df = start_of_season_df.groupby('h_name').mean().reset_index()
away_start_df = start_of_season_df.groupby('v_name').mean().reset_index()

In [29]:
away_last30_mean = away_start_df
home_last30_mean = home_start_df

In [30]:
home_start_df

,h_name,date,v_score,h_score,attendance,v_at_bats,v_hits,v_rbi,v_strikeouts,v_caught_stealing,v_grounded_into_double,v_first_catcher_interference,v_left_on_base,v_pitchers_used,v_individual_earned_runs,v_team_earned_runs,v_putouts,v_assists,v_errors,v_double_plays,v_triple_plays,h_at_bats,h_hits,h_rbi,h_strikeouts,h_caught_stealing,h_grounded_into_double,h_first_catcher_interference,h_left_on_base,h_pitchers_used,h_individual_earned_runs,h_team_earned_runs,h_putouts,h_assists,h_errors,h_double_plays,h_triple_plays,Run_dif,Win,h_xbh,v_xbh,v_pitcher_allowed_on_base,h_pitcher_allowed_on_base,v_advanced_runner,h_advanced_runner,v_pitcher_allowed_runner_advance,h_pitcher_allowed_runner_advance
0,ANA,2.016048e+07,4.733333,4.066667,38136.466667,34.733333,8.900000,4.566667,8.200000,0.300000,0.700000,0.000000,6.466667,3.600000,3.766667,3.766667,26.366667,11.433333,0.533333,1.100000,0.0,32.466667,7.766667,3.866667,6.133333,0.200000,1.000000,0.066667,5.966667,4.400000,4.366667,4.366667,27.500000,9.033333,0.633333,0.900000,0.000000,-0.666667,0.400000,2.366667,3.000000,3.500000,3.566667,1.066667,0.766667,0.433333,0.433333
1,ARI,2.016048e+07,5.848485,4.606061,25296.181818,35.545455,9.939394,5.606061,7.606061,0.151515,1.090909,0.000000,6.969697,4.000000,4.303030,4.272727,26.515152,10.818182,0.515152,0.939394,0.0,34.424242,9.030303,4.393939,8.939394,0.121212,0.818182,0.000000,7.060606,4.303030,5.696970,5.696970,27.363636,11.151515,0.393939,1.121212,0.000000,-1.242424,0.303030,3.303030,3.787879,3.515152,4.484848,0.606061,1.181818,0.575758,0.424242
2,ATL,2.016048e+07,4.733333,3.066667,22843.266667,35.433333,8.633333,4.400000,8.333333,0.266667,0.600000,0.000000,7.433333,4.500000,2.766667,2.766667,27.500000,10.033333,0.766667,0.966667,0.0,33.533333,7.600000,2.900000,8.166667,0.200000,0.833333,0.000000,7.266667,4.800000,4.266667,4.266667,28.200000,10.533333,0.866667,0.866667,0.000000,-1.666667,0.233333,1.566667,3.133333,3.833333,4.733333,1.400000,1.133333,0.366667,0.700000
3,BAL,2.016050e+07,3.942857,4.885714,23769.257143,34.800000,9.114286,3.800000,7.914286,0.314286,0.800000,0.028571,7.514286,3.685714,4.485714,4.485714,25.257143,9.142857,0.457143,0.914286,0.0,32.714286,8.514286,4.571429,7.400000,0.057143,0.828571,0.000000,6.514286,3.600000,3.742857,3.742857,27.171429,10.057143,0.485714,0.971429,0.000000,0.942857,0.685714,3.171429,3.085714,3.771429,3.600000,0.571429,0.428571,0.571429,0.400000
4,BOS,2.016048e+07,4.580645,6.419355,35161.096774,34.419355,8.354839,4.322581,8.870968,0.225806,0.709677,0.000000,6.870968,4.032258,6.193548,6.193548,25.516129,9.903226,0.612903,0.806452,0.0,35.161290,10.580645,6.161290,6.838710,0.000000,0.741935,0.000000,7.483871,4.032258,4.096774,4.096774,27.290323,8.903226,0.516129,0.903226,0.000000,1.838710,0.612903,4.354839,3.032258,4.354839,4.161290,0.709677,0.870968,0.774194,0.903226
5,CHA,2.016050e+07,3.896552,3.655172,19651.379310,33.965517,8.344828,3.827586,7.620690,0.172414,0.896552,0.000000,6.344828,3.551724,3.482759,3.482759,25.896552,9.448276,0.482759,0.896552,0.0,32.068966,7.586207,3.413793,7.551724,0.241379,0.655172,0.000000,6.965517,3.931034,3.689655,3.655172,27.413793,11.137931,0.344828,1.068966,0.068966,-0.241379,0.517241,2.689655,2.482759,4.241379,3.206897,1.206897,0.862069,0.517241,0.482759
6,CHN,2.016050e+07,2.800000,4.833333,38349.633333,32.233333,6.366667,2.533333,9.566667,0.166667,0.900000,0.000000,6.166667,4.000000,4.566667,4.566667,25.333333,8.200000,0.700000,0.566667,0.0,31.666667,7.566667,4.700000,8.333333,0.233333,0.500000,0.000000,7.066667,4.066667,2.533333,2.533333,27.500000,11.033333,0.633333,1.033333,0.000000,2.033333,0.733333,2.733333,1.933333,5.433333,3.966667,1.366667,0.900000,0.633333,0.600000
7,CIN,2.016049e+07,5.441176,4.647059,22427.470588,35.029412,9.235294,5.264706,7.088235,0.411765,0.500000,0.000000,7.352941,3.852941,4.500000,4.500000,26.058824,9.382353,0.588235,0.735294,0.0,32.235294,7.794118,4.382353,8.294118,0.264706,0.647059,0.000000,4.588235,4.205882,4.970588,4.941176,27.352941

# Start of Season Functions

In [31]:
def home_start_szn_rd(i):
        if i == 'ANA':
            return home_start_df['Run_dif'].loc[0]
        if i == 'ARI':
            return home_start_df['Run_dif'].loc[1]
        if i == 'ATL':
            return home_start_df['Run_dif'].loc[2]
        if i == 'BAL':
            return home_start_df['Run_dif'].loc[3]
        if i == 'BOS':
            return home_start_df['Run_dif'].loc[4]
        if i == 'CHA':
            return home_start_df['Run_dif'].loc[5]
        if i == 'CHN':
            return home_start_df['Run_dif'].loc[6]
        if i == 'CIN':
            return home_start_df['Run_dif'].loc[7]
        if i == 'CLE':
            return home_start_df['Run_dif'].loc[8]
        if i == 'COL':
            return home_start_df['Run_dif'].loc[9]
        if i == 'DET':
            return home_start_df['Run_dif'].loc[10]
        if i == 'HOU':
            return home_start_df['Run_dif'].loc[11]
        if i == 'KCA':
            return home_start_df['Run_dif'].loc[12]
        if i == 'LAN':
            return home_start_df['Run_dif'].loc[13]
        if i == 'MIA':
            return home_start_df['Run_dif'].loc[14]
        if i == 'MIL':
            return home_start_df['Run_dif'].loc[15]
        if i == 'MIN':
            return home_start_df['Run_dif'].loc[16]
        if i == 'NYA':
            return home_start_df['Run_dif'].loc[17]
        if i == 'NYN':
            return home_start_df['Run_dif'].loc[18]
        if i == 'OAK':
            return home_start_df['Run_dif'].loc[19]
        if i == 'PHI':
            return home_start_df['Run_dif'].loc[20]
        if i == 'PIT':
            return home_start_df['Run_dif'].loc[21]
        if i == 'SDN':
            return home_start_df['Run_dif'].loc[22]
        if i == 'SEA':
            return home_start_df['Run_dif'].loc[23]
        if i == 'SFN':
            return home_start_df['Run_dif'].loc[24]
        if i == 'SLN':
            return home_start_df['Run_dif'].loc[25]
        if i == 'TBA':
            return home_start_df['Run_dif'].loc[26]
        if i == 'TEX':
            return home_start_df['Run_dif'].loc[27]
        if i == 'TOR':
            return home_start_df['Run_dif'].loc[28]
        if i == 'WAS':
            return home_start_df['Run_dif'].loc[29]

In [32]:
def away_start_szn_rd(i):
        if i == 'ANA':
            return away_start_df['Run_dif'].loc[0]
        if i == 'ARI':
            return away_start_df['Run_dif'].loc[1]
        if i == 'ATL':
            return away_start_df['Run_dif'].loc[2]
        if i == 'BAL':
            return away_start_df['Run_dif'].loc[3]
        if i == 'BOS':
            return away_start_df['Run_dif'].loc[4]
        if i == 'CHA':
            return away_start_df['Run_dif'].loc[5]
        if i == 'CHN':
            return away_start_df['Run_dif'].loc[6]
        if i == 'CIN':
            return away_start_df['Run_dif'].loc[7]
        if i == 'CLE':
            return away_start_df['Run_dif'].loc[8]
        if i == 'COL':
            return away_start_df['Run_dif'].loc[9]
        if i == 'DET':
            return away_start_df['Run_dif'].loc[10]
        if i == 'HOU':
            return away_start_df['Run_dif'].loc[11]
        if i == 'KCA':
            return away_start_df['Run_dif'].loc[12]
        if i == 'LAN':
            return away_start_df['Run_dif'].loc[13]
        if i == 'MIA':
            return away_start_df['Run_dif'].loc[14]
        if i == 'MIL':
            return away_start_df['Run_dif'].loc[15]
        if i == 'MIN':
            return away_start_df['Run_dif'].loc[16]
        if i == 'NYA':
            return away_start_df['Run_dif'].loc[17]
        if i == 'NYN':
            return away_start_df['Run_dif'].loc[18]
        if i == 'OAK':
            return away_start_df['Run_dif'].loc[19]
        if i == 'PHI':
            return away_start_df['Run_dif'].loc[20]
        if i == 'PIT':
            return away_start_df['Run_dif'].loc[21]
        if i == 'SDN':
            return away_start_df['Run_dif'].loc[22]
        if i == 'SEA':
            return away_start_df['Run_dif'].loc[23]
        if i == 'SFN':
            return away_start_df['Run_dif'].loc[24]
        if i == 'SLN':
            return away_start_df['Run_dif'].loc[25]
        if i == 'TBA':
            return away_start_df['Run_dif'].loc[26]
        if i == 'TEX':
            return away_start_df['Run_dif'].loc[27]
        if i == 'TOR':
            return away_start_df['Run_dif'].loc[28]
        if i == 'WAS':
            return away_start_df['Run_dif'].loc[29]

In [33]:
def home_start_szn_W(i):
        if i == 'ANA':
            return home_start_df['Win'].loc[0]
        if i == 'ARI':
            return home_start_df['Win'].loc[1]
        if i == 'ATL':
            return home_start_df['Win'].loc[2]
        if i == 'BAL':
            return home_start_df['Win'].loc[3]
        if i == 'BOS':
            return home_start_df['Win'].loc[4]
        if i == 'CHA':
            return home_start_df['Win'].loc[5]
        if i == 'CHN':
            return home_start_df['Win'].loc[6]
        if i == 'CIN':
            return home_start_df['Win'].loc[7]
        if i == 'CLE':
            return home_start_df['Win'].loc[8]
        if i == 'COL':
            return home_start_df['Win'].loc[9]
        if i == 'DET':
            return home_start_df['Win'].loc[10]
        if i == 'HOU':
            return home_start_df['Win'].loc[11]
        if i == 'KCA':
            return home_start_df['Win'].loc[12]
        if i == 'LAN':
            return home_start_df['Win'].loc[13]
        if i == 'MIA':
            return home_start_df['Win'].loc[14]
        if i == 'MIL':
            return home_start_df['Win'].loc[15]
        if i == 'MIN':
            return home_start_df['Win'].loc[16]
        if i == 'NYA':
            return home_start_df['Win'].loc[17]
        if i == 'NYN':
            return home_start_df['Win'].loc[18]
        if i == 'OAK':
            return home_start_df['Win'].loc[19]
        if i == 'PHI':
            return home_start_df['Win'].loc[20]
        if i == 'PIT':
            return home_start_df['Win'].loc[21]
        if i == 'SDN':
            return home_start_df['Win'].loc[22]
        if i == 'SEA':
            return home_start_df['Win'].loc[23]
        if i == 'SFN':
            return home_start_df['Win'].loc[24]
        if i == 'SLN':
            return home_start_df['Win'].loc[25]
        if i == 'TBA':
            return home_start_df['Win'].loc[26]
        if i == 'TEX':
            return home_start_df['Win'].loc[27]
        if i == 'TOR':
            return home_start_df['Win'].loc[28]
        if i == 'WAS':
            return home_start_df['Win'].loc[29]

In [34]:
def away_start_szn_W(i):
        if i == 'ANA':
            return away_start_df['Win'].loc[0]
        if i == 'ARI':
            return away_start_df['Win'].loc[1]
        if i == 'ATL':
            return away_start_df['Win'].loc[2]
        if i == 'BAL':
            return away_start_df['Win'].loc[3]
        if i == 'BOS':
            return away_start_df['Win'].loc[4]
        if i == 'CHA':
            return away_start_df['Win'].loc[5]
        if i == 'CHN':
            return away_start_df['Win'].loc[6]
        if i == 'CIN':
            return away_start_df['Win'].loc[7]
        if i == 'CLE':
            return away_start_df['Win'].loc[8]
        if i == 'COL':
            return away_start_df['Win'].loc[9]
        if i == 'DET':
            return away_start_df['Win'].loc[10]
        if i == 'HOU':
            return away_start_df['Win'].loc[11]
        if i == 'KCA':
            return away_start_df['Win'].loc[12]
        if i == 'LAN':
            return away_start_df['Win'].loc[13]
        if i == 'MIA':
            return away_start_df['Win'].loc[14]
        if i == 'MIL':
            return away_start_df['Win'].loc[15]
        if i == 'MIN':
            return away_start_df['Win'].loc[16]
        if i == 'NYA':
            return away_start_df['Win'].loc[17]
        if i == 'NYN':
            return away_start_df['Win'].loc[18]
        if i == 'OAK':
            return away_start_df['Win'].loc[19]
        if i == 'PHI':
            return away_start_df['Win'].loc[20]
        if i == 'PIT':
            return away_start_df['Win'].loc[21]
        if i == 'SDN':
            return away_start_df['Win'].loc[22]
        if i == 'SEA':
            return away_start_df['Win'].loc[23]
        if i == 'SFN':
            return away_start_df['Win'].loc[24]
        if i == 'SLN':
            return away_start_df['Win'].loc[25]
        if i == 'TBA':
            return away_start_df['Win'].loc[26]
        if i == 'TEX':
            return away_start_df['Win'].loc[27]
        if i == 'TOR':
            return away_start_df['Win'].loc[28]
        if i == 'WAS':
            return away_start_df['Win'].loc[29]

In [35]:
def home_start_szn_hits(i):
        if i == 'ANA':
            return home_start_df['h_hits'].loc[0]
        if i == 'ARI':
            return home_start_df['h_hits'].loc[1]
        if i == 'ATL':
            return home_start_df['h_hits'].loc[2]
        if i == 'BAL':
            return home_start_df['h_hits'].loc[3]
        if i == 'BOS':
            return home_start_df['h_hits'].loc[4]
        if i == 'CHA':
            return home_start_df['h_hits'].loc[5]
        if i == 'CHN':
            return home_start_df['h_hits'].loc[6]
        if i == 'CIN':
            return home_start_df['h_hits'].loc[7]
        if i == 'CLE':
            return home_start_df['h_hits'].loc[8]
        if i == 'COL':
            return home_start_df['h_hits'].loc[9]
        if i == 'DET':
            return home_start_df['h_hits'].loc[10]
        if i == 'HOU':
            return home_start_df['h_hits'].loc[11]
        if i == 'KCA':
            return home_start_df['h_hits'].loc[12]
        if i == 'LAN':
            return home_start_df['h_hits'].loc[13]
        if i == 'MIA':
            return home_start_df['h_hits'].loc[14]
        if i == 'MIL':
            return home_start_df['h_hits'].loc[15]
        if i == 'MIN':
            return home_start_df['h_hits'].loc[16]
        if i == 'NYA':
            return home_start_df['h_hits'].loc[17]
        if i == 'NYN':
            return home_start_df['h_hits'].loc[18]
        if i == 'OAK':
            return home_start_df['h_hits'].loc[19]
        if i == 'PHI':
            return home_start_df['h_hits'].loc[20]
        if i == 'PIT':
            return home_start_df['h_hits'].loc[21]
        if i == 'SDN':
            return home_start_df['h_hits'].loc[22]
        if i == 'SEA':
            return home_start_df['h_hits'].loc[23]
        if i == 'SFN':
            return home_start_df['h_hits'].loc[24]
        if i == 'SLN':
            return home_start_df['h_hits'].loc[25]
        if i == 'TBA':
            return home_start_df['h_hits'].loc[26]
        if i == 'TEX':
            return home_start_df['h_hits'].loc[27]
        if i == 'TOR':
            return home_start_df['h_hits'].loc[28]
        if i == 'WAS':
            return home_start_df['h_hits'].loc[29]

In [36]:
def away_start_szn_hits(i):
        if i == 'ANA':
            return away_start_df['v_hits'].loc[0]
        if i == 'ARI':
            return away_start_df['v_hits'].loc[1]
        if i == 'ATL':
            return away_start_df['v_hits'].loc[2]
        if i == 'BAL':
            return away_start_df['v_hits'].loc[3]
        if i == 'BOS':
            return away_start_df['v_hits'].loc[4]
        if i == 'CHA':
            return away_start_df['v_hits'].loc[5]
        if i == 'CHN':
            return away_start_df['v_hits'].loc[6]
        if i == 'CIN':
            return away_start_df['v_hits'].loc[7]
        if i == 'CLE':
            return away_start_df['v_hits'].loc[8]
        if i == 'COL':
            return away_start_df['v_hits'].loc[9]
        if i == 'DET':
            return away_start_df['v_hits'].loc[10]
        if i == 'HOU':
            return away_start_df['v_hits'].loc[11]
        if i == 'KCA':
            return away_start_df['v_hits'].loc[12]
        if i == 'LAN':
            return away_start_df['v_hits'].loc[13]
        if i == 'MIA':
            return away_start_df['v_hits'].loc[14]
        if i == 'MIL':
            return away_start_df['v_hits'].loc[15]
        if i == 'MIN':
            return away_start_df['v_hits'].loc[16]
        if i == 'NYA':
            return away_start_df['v_hits'].loc[17]
        if i == 'NYN':
            return away_start_df['v_hits'].loc[18]
        if i == 'OAK':
            return away_start_df['v_hits'].loc[19]
        if i == 'PHI':
            return away_start_df['v_hits'].loc[20]
        if i == 'PIT':
            return away_start_df['v_hits'].loc[21]
        if i == 'SDN':
            return away_start_df['v_hits'].loc[22]
        if i == 'SEA':
            return away_start_df['v_hits'].loc[23]
        if i == 'SFN':
            return away_start_df['v_hits'].loc[24]
        if i == 'SLN':
            return away_start_df['v_hits'].loc[25]
        if i == 'TBA':
            return away_start_df['v_hits'].loc[26]
        if i == 'TEX':
            return away_start_df['v_hits'].loc[27]
        if i == 'TOR':
            return away_start_df['v_hits'].loc[28]
        if i == 'WAS':
            return away_start_df['v_hits'].loc[29]

In [37]:
def home_start_szn_hits_allowed(i):
        if i == 'ANA':
            return home_start_df['v_hits'].loc[0]
        if i == 'ARI':
            return home_start_df['v_hits'].loc[1]
        if i == 'ATL':
            return home_start_df['v_hits'].loc[2]
        if i == 'BAL':
            return home_start_df['v_hits'].loc[3]
        if i == 'BOS':
            return home_start_df['v_hits'].loc[4]
        if i == 'CHA':
            return home_start_df['v_hits'].loc[5]
        if i == 'CHN':
            return home_start_df['v_hits'].loc[6]
        if i == 'CIN':
            return home_start_df['v_hits'].loc[7]
        if i == 'CLE':
            return home_start_df['v_hits'].loc[8]
        if i == 'COL':
            return home_start_df['v_hits'].loc[9]
        if i == 'DET':
            return home_start_df['v_hits'].loc[10]
        if i == 'HOU':
            return home_start_df['v_hits'].loc[11]
        if i == 'KCA':
            return home_start_df['v_hits'].loc[12]
        if i == 'LAN':
            return home_start_df['v_hits'].loc[13]
        if i == 'MIA':
            return home_start_df['v_hits'].loc[14]
        if i == 'MIL':
            return home_start_df['v_hits'].loc[15]
        if i == 'MIN':
            return home_start_df['v_hits'].loc[16]
        if i == 'NYA':
            return home_start_df['v_hits'].loc[17]
        if i == 'NYN':
            return home_start_df['v_hits'].loc[18]
        if i == 'OAK':
            return home_start_df['v_hits'].loc[19]
        if i == 'PHI':
            return home_start_df['v_hits'].loc[20]
        if i == 'PIT':
            return home_start_df['v_hits'].loc[21]
        if i == 'SDN':
            return home_start_df['v_hits'].loc[22]
        if i == 'SEA':
            return home_start_df['v_hits'].loc[23]
        if i == 'SFN':
            return home_start_df['v_hits'].loc[24]
        if i == 'SLN':
            return home_start_df['v_hits'].loc[25]
        if i == 'TBA':
            return home_start_df['v_hits'].loc[26]
        if i == 'TEX':
            return home_start_df['v_hits'].loc[27]
        if i == 'TOR':
            return home_start_df['v_hits'].loc[28]
        if i == 'WAS':
            return home_start_df['v_hits'].loc[29]

In [38]:
def away_start_szn_hits_allowed(i):
        if i == 'ANA':
            return away_start_df['h_hits'].loc[0]
        if i == 'ARI':
            return away_start_df['h_hits'].loc[1]
        if i == 'ATL':
            return away_start_df['h_hits'].loc[2]
        if i == 'BAL':
            return away_start_df['h_hits'].loc[3]
        if i == 'BOS':
            return away_start_df['h_hits'].loc[4]
        if i == 'CHA':
            return away_start_df['h_hits'].loc[5]
        if i == 'CHN':
            return away_start_df['h_hits'].loc[6]
        if i == 'CIN':
            return away_start_df['h_hits'].loc[7]
        if i == 'CLE':
            return away_start_df['h_hits'].loc[8]
        if i == 'COL':
            return away_start_df['h_hits'].loc[9]
        if i == 'DET':
            return away_start_df['h_hits'].loc[10]
        if i == 'HOU':
            return away_start_df['h_hits'].loc[11]
        if i == 'KCA':
            return away_start_df['h_hits'].loc[12]
        if i == 'LAN':
            return away_start_df['h_hits'].loc[13]
        if i == 'MIA':
            return away_start_df['h_hits'].loc[14]
        if i == 'MIL':
            return away_start_df['h_hits'].loc[15]
        if i == 'MIN':
            return away_start_df['h_hits'].loc[16]
        if i == 'NYA':
            return away_start_df['h_hits'].loc[17]
        if i == 'NYN':
            return away_start_df['h_hits'].loc[18]
        if i == 'OAK':
            return away_start_df['h_hits'].loc[19]
        if i == 'PHI':
            return away_start_df['h_hits'].loc[20]
        if i == 'PIT':
            return away_start_df['h_hits'].loc[21]
        if i == 'SDN':
            return away_start_df['h_hits'].loc[22]
        if i == 'SEA':
            return away_start_df['h_hits'].loc[23]
        if i == 'SFN':
            return away_start_df['h_hits'].loc[24]
        if i == 'SLN':
            return away_start_df['h_hits'].loc[25]
        if i == 'TBA':
            return away_start_df['h_hits'].loc[26]
        if i == 'TEX':
            return away_start_df['h_hits'].loc[27]
        if i == 'TOR':
            return away_start_df['h_hits'].loc[28]
        if i == 'WAS':
            return away_start_df['h_hits'].loc[29]

In [39]:
def home_start_szn_xbh(i):
        if i == 'ANA':
            return home_start_df['h_xbh'].loc[0]
        if i == 'ARI':
            return home_start_df['h_xbh'].loc[1]
        if i == 'ATL':
            return home_start_df['h_xbh'].loc[2]
        if i == 'BAL':
            return home_start_df['h_xbh'].loc[3]
        if i == 'BOS':
            return home_start_df['h_xbh'].loc[4]
        if i == 'CHA':
            return home_start_df['h_xbh'].loc[5]
        if i == 'CHN':
            return home_start_df['h_xbh'].loc[6]
        if i == 'CIN':
            return home_start_df['h_xbh'].loc[7]
        if i == 'CLE':
            return home_start_df['h_xbh'].loc[8]
        if i == 'COL':
            return home_start_df['h_xbh'].loc[9]
        if i == 'DET':
            return home_start_df['h_xbh'].loc[10]
        if i == 'HOU':
            return home_start_df['h_xbh'].loc[11]
        if i == 'KCA':
            return home_start_df['h_xbh'].loc[12]
        if i == 'LAN':
            return home_start_df['h_xbh'].loc[13]
        if i == 'MIA':
            return home_start_df['h_xbh'].loc[14]
        if i == 'MIL':
            return home_start_df['h_xbh'].loc[15]
        if i == 'MIN':
            return home_start_df['h_xbh'].loc[16]
        if i == 'NYA':
            return home_start_df['h_xbh'].loc[17]
        if i == 'NYN':
            return home_start_df['h_xbh'].loc[18]
        if i == 'OAK':
            return home_start_df['h_xbh'].loc[19]
        if i == 'PHI':
            return home_start_df['h_xbh'].loc[20]
        if i == 'PIT':
            return home_start_df['h_xbh'].loc[21]
        if i == 'SDN':
            return home_start_df['h_xbh'].loc[22]
        if i == 'SEA':
            return home_start_df['h_xbh'].loc[23]
        if i == 'SFN':
            return home_start_df['h_xbh'].loc[24]
        if i == 'SLN':
            return home_start_df['h_xbh'].loc[25]
        if i == 'TBA':
            return home_start_df['h_xbh'].loc[26]
        if i == 'TEX':
            return home_start_df['h_xbh'].loc[27]
        if i == 'TOR':
            return home_start_df['h_xbh'].loc[28]
        if i == 'WAS':
            return home_start_df['h_xbh'].loc[29]

In [40]:
def away_start_szn_xbh(i):
        if i == 'ANA':
            return away_start_df['v_xbh'].loc[0]
        if i == 'ARI':
            return away_start_df['v_xbh'].loc[1]
        if i == 'ATL':
            return away_start_df['v_xbh'].loc[2]
        if i == 'BAL':
            return away_start_df['v_xbh'].loc[3]
        if i == 'BOS':
            return away_start_df['v_xbh'].loc[4]
        if i == 'CHA':
            return away_start_df['v_xbh'].loc[5]
        if i == 'CHN':
            return away_start_df['v_xbh'].loc[6]
        if i == 'CIN':
            return away_start_df['v_xbh'].loc[7]
        if i == 'CLE':
            return away_start_df['v_xbh'].loc[8]
        if i == 'COL':
            return away_start_df['v_xbh'].loc[9]
        if i == 'DET':
            return away_start_df['v_xbh'].loc[10]
        if i == 'HOU':
            return away_start_df['v_xbh'].loc[11]
        if i == 'KCA':
            return away_start_df['v_xbh'].loc[12]
        if i == 'LAN':
            return away_start_df['v_xbh'].loc[13]
        if i == 'MIA':
            return away_start_df['v_xbh'].loc[14]
        if i == 'MIL':
            return away_start_df['v_xbh'].loc[15]
        if i == 'MIN':
            return away_start_df['v_xbh'].loc[16]
        if i == 'NYA':
            return away_start_df['v_xbh'].loc[17]
        if i == 'NYN':
            return away_start_df['v_xbh'].loc[18]
        if i == 'OAK':
            return away_start_df['v_xbh'].loc[19]
        if i == 'PHI':
            return away_start_df['v_xbh'].loc[20]
        if i == 'PIT':
            return away_start_df['v_xbh'].loc[21]
        if i == 'SDN':
            return away_start_df['v_xbh'].loc[22]
        if i == 'SEA':
            return away_start_df['v_xbh'].loc[23]
        if i == 'SFN':
            return away_start_df['v_xbh'].loc[24]
        if i == 'SLN':
            return away_start_df['v_xbh'].loc[25]
        if i == 'TBA':
            return away_start_df['v_xbh'].loc[26]
        if i == 'TEX':
            return away_start_df['v_xbh'].loc[27]
        if i == 'TOR':
            return away_start_df['v_xbh'].loc[28]
        if i == 'WAS':
            return away_start_df['v_xbh'].loc[29]

In [41]:
def home_last_30_days_xbh_allowed(i):
        if i == 'ANA':
            return home_last30_mean['v_xbh'].loc[0]
        if i == 'ARI':
            return home_last30_mean['v_xbh'].loc[1]
        if i == 'ATL':
            return home_last30_mean['v_xbh'].loc[2]
        if i == 'BAL':
            return home_last30_mean['v_xbh'].loc[3]
        if i == 'BOS':
            return home_last30_mean['v_xbh'].loc[4]
        if i == 'CHA':
            return home_last30_mean['v_xbh'].loc[5]
        if i == 'CHN':
            return home_last30_mean['v_xbh'].loc[6]
        if i == 'CIN':
            return home_last30_mean['v_xbh'].loc[7]
        if i == 'CLE':
            return home_last30_mean['v_xbh'].loc[8]
        if i == 'COL':
            return home_last30_mean['v_xbh'].loc[9]
        if i == 'DET':
            return home_last30_mean['v_xbh'].loc[10]
        if i == 'HOU':
            return home_last30_mean['v_xbh'].loc[11]
        if i == 'KCA':
            return home_last30_mean['v_xbh'].loc[12]
        if i == 'LAN':
            return home_last30_mean['v_xbh'].loc[13]
        if i == 'MIA':
            return home_last30_mean['v_xbh'].loc[14]
        if i == 'MIL':
            return home_last30_mean['v_xbh'].loc[15]
        if i == 'MIN':
            return home_last30_mean['v_xbh'].loc[16]
        if i == 'NYA':
            return home_last30_mean['v_xbh'].loc[17]
        if i == 'NYN':
            return home_last30_mean['v_xbh'].loc[18]
        if i == 'OAK':
            return home_last30_mean['v_xbh'].loc[19]
        if i == 'PHI':
            return home_last30_mean['v_xbh'].loc[20]
        if i == 'PIT':
            return home_last30_mean['v_xbh'].loc[21]
        if i == 'SDN':
            return home_last30_mean['v_xbh'].loc[22]
        if i == 'SEA':
            return home_last30_mean['v_xbh'].loc[23]
        if i == 'SFN':
            return home_last30_mean['v_xbh'].loc[24]
        if i == 'SLN':
            return home_last30_mean['v_xbh'].loc[25]
        if i == 'TBA':
            return home_last30_mean['v_xbh'].loc[26]
        if i == 'TEX':
            return home_last30_mean['v_xbh'].loc[27]
        if i == 'TOR':
            return home_last30_mean['v_xbh'].loc[28]
        if i == 'WAS':
            return home_last30_mean['v_xbh'].loc[29]

In [42]:
def away_last_30_days_xbh_allowed(i):
        if i == 'ANA':
            return away_last30_mean['h_xbh'].loc[0]
        if i == 'ARI':
            return away_last30_mean['h_xbh'].loc[1]
        if i == 'ATL':
            return away_last30_mean['h_xbh'].loc[2]
        if i == 'BAL':
            return away_last30_mean['h_xbh'].loc[3]
        if i == 'BOS':
            return away_last30_mean['h_xbh'].loc[4]
        if i == 'CHA':
            return away_last30_mean['h_xbh'].loc[5]
        if i == 'CHN':
            return away_last30_mean['h_xbh'].loc[6]
        if i == 'CIN':
            return away_last30_mean['h_xbh'].loc[7]
        if i == 'CLE':
            return away_last30_mean['h_xbh'].loc[8]
        if i == 'COL':
            return away_last30_mean['h_xbh'].loc[9]
        if i == 'DET':
            return away_last30_mean['h_xbh'].loc[10]
        if i == 'HOU':
            return away_last30_mean['h_xbh'].loc[11]
        if i == 'KCA':
            return away_last30_mean['h_xbh'].loc[12]
        if i == 'LAN':
            return away_last30_mean['h_xbh'].loc[13]
        if i == 'MIA':
            return away_last30_mean['h_xbh'].loc[14]
        if i == 'MIL':
            return away_last30_mean['h_xbh'].loc[15]
        if i == 'MIN':
            return away_last30_mean['h_xbh'].loc[16]
        if i == 'NYA':
            return away_last30_mean['h_xbh'].loc[17]
        if i == 'NYN':
            return away_last30_mean['h_xbh'].loc[18]
        if i == 'OAK':
            return away_last30_mean['h_xbh'].loc[19]
        if i == 'PHI':
            return away_last30_mean['h_xbh'].loc[20]
        if i == 'PIT':
            return away_last30_mean['h_xbh'].loc[21]
        if i == 'SDN':
            return away_last30_mean['h_xbh'].loc[22]
        if i == 'SEA':
            return away_last30_mean['h_xbh'].loc[23]
        if i == 'SFN':
            return away_last30_mean['h_xbh'].loc[24]
        if i == 'SLN':
            return away_last30_mean['h_xbh'].loc[25]
        if i == 'TBA':
            return away_last30_mean['h_xbh'].loc[26]
        if i == 'TEX':
            return away_last30_mean['h_xbh'].loc[27]
        if i == 'TOR':
            return away_last30_mean['h_xbh'].loc[28]
        if i == 'WAS':
            return away_last30_mean['h_xbh'].loc[29]

In [43]:
def home_last_30_days_on_base_by_pitcher(i):
        if i == 'ANA':
            return home_last30_mean['v_pitcher_allowed_on_base'].loc[0]
        if i == 'ARI':
            return home_last30_mean['v_pitcher_allowed_on_base'].loc[1]
        if i == 'ATL':
            return home_last30_mean['v_pitcher_allowed_on_base'].loc[2]
        if i == 'BAL':
            return home_last30_mean['v_pitcher_allowed_on_base'].loc[3]
        if i == 'BOS':
            return home_last30_mean['v_pitcher_allowed_on_base'].loc[4]
        if i == 'CHA':
            return home_last30_mean['v_pitcher_allowed_on_base'].loc[5]
        if i == 'CHN':
            return home_last30_mean['v_pitcher_allowed_on_base'].loc[6]
        if i == 'CIN':
            return home_last30_mean['v_pitcher_allowed_on_base'].loc[7]
        if i == 'CLE':
            return home_last30_mean['v_pitcher_allowed_on_base'].loc[8]
        if i == 'COL':
            return home_last30_mean['v_pitcher_allowed_on_base'].loc[9]
        if i == 'DET':
            return home_last30_mean['v_pitcher_allowed_on_base'].loc[10]
        if i == 'HOU':
            return home_last30_mean['v_pitcher_allowed_on_base'].loc[11]
        if i == 'KCA':
            return home_last30_mean['v_pitcher_allowed_on_base'].loc[12]
        if i == 'LAN':
            return home_last30_mean['v_pitcher_allowed_on_base'].loc[13]
        if i == 'MIA':
            return home_last30_mean['v_pitcher_allowed_on_base'].loc[14]
        if i == 'MIL':
            return home_last30_mean['v_pitcher_allowed_on_base'].loc[15]
        if i == 'MIN':
            return home_last30_mean['v_pitcher_allowed_on_base'].loc[16]
        if i == 'NYA':
            return home_last30_mean['v_pitcher_allowed_on_base'].loc[17]
        if i == 'NYN':
            return home_last30_mean['v_pitcher_allowed_on_base'].loc[18]
        if i == 'OAK':
            return home_last30_mean['v_pitcher_allowed_on_base'].loc[19]
        if i == 'PHI':
            return home_last30_mean['v_pitcher_allowed_on_base'].loc[20]
        if i == 'PIT':
            return home_last30_mean['v_pitcher_allowed_on_base'].loc[21]
        if i == 'SDN':
            return home_last30_mean['v_pitcher_allowed_on_base'].loc[22]
        if i == 'SEA':
            return home_last30_mean['v_pitcher_allowed_on_base'].loc[23]
        if i == 'SFN':
            return home_last30_mean['v_pitcher_allowed_on_base'].loc[24]
        if i == 'SLN':
            return home_last30_mean['v_pitcher_allowed_on_base'].loc[25]
        if i == 'TBA':
            return home_last30_mean['v_pitcher_allowed_on_base'].loc[26]
        if i == 'TEX':
            return home_last30_mean['v_pitcher_allowed_on_base'].loc[27]
        if i == 'TOR':
            return home_last30_mean['v_pitcher_allowed_on_base'].loc[28]
        if i == 'WAS':
            return home_last30_mean['v_pitcher_allowed_on_base'].loc[29]

In [44]:
def away_last_30_days_on_base_by_pitcher(i):
        if i == 'ANA':
            return away_last30_mean['h_pitcher_allowed_on_base'].loc[0]
        if i == 'ARI':
            return away_last30_mean['h_pitcher_allowed_on_base'].loc[1]
        if i == 'ATL':
            return away_last30_mean['h_pitcher_allowed_on_base'].loc[2]
        if i == 'BAL':
            return away_last30_mean['h_pitcher_allowed_on_base'].loc[3]
        if i == 'BOS':
            return away_last30_mean['h_pitcher_allowed_on_base'].loc[4]
        if i == 'CHA':
            return away_last30_mean['h_pitcher_allowed_on_base'].loc[5]
        if i == 'CHN':
            return away_last30_mean['h_pitcher_allowed_on_base'].loc[6]
        if i == 'CIN':
            return away_last30_mean['h_pitcher_allowed_on_base'].loc[7]
        if i == 'CLE':
            return away_last30_mean['h_pitcher_allowed_on_base'].loc[8]
        if i == 'COL':
            return away_last30_mean['h_pitcher_allowed_on_base'].loc[9]
        if i == 'DET':
            return away_last30_mean['h_pitcher_allowed_on_base'].loc[10]
        if i == 'HOU':
            return away_last30_mean['h_pitcher_allowed_on_base'].loc[11]
        if i == 'KCA':
            return away_last30_mean['h_pitcher_allowed_on_base'].loc[12]
        if i == 'LAN':
            return away_last30_mean['h_pitcher_allowed_on_base'].loc[13]
        if i == 'MIA':
            return away_last30_mean['h_pitcher_allowed_on_base'].loc[14]
        if i == 'MIL':
            return away_last30_mean['h_pitcher_allowed_on_base'].loc[15]
        if i == 'MIN':
            return away_last30_mean['h_pitcher_allowed_on_base'].loc[16]
        if i == 'NYA':
            return away_last30_mean['h_pitcher_allowed_on_base'].loc[17]
        if i == 'NYN':
            return away_last30_mean['h_pitcher_allowed_on_base'].loc[18]
        if i == 'OAK':
            return away_last30_mean['h_pitcher_allowed_on_base'].loc[19]
        if i == 'PHI':
            return away_last30_mean['h_pitcher_allowed_on_base'].loc[20]
        if i == 'PIT':
            return away_last30_mean['h_pitcher_allowed_on_base'].loc[21]
        if i == 'SDN':
            return away_last30_mean['h_pitcher_allowed_on_base'].loc[22]
        if i == 'SEA':
            return away_last30_mean['h_pitcher_allowed_on_base'].loc[23]
        if i == 'SFN':
            return away_last30_mean['h_pitcher_allowed_on_base'].loc[24]
        if i == 'SLN':
            return away_last30_mean['h_pitcher_allowed_on_base'].loc[25]
        if i == 'TBA':
            return away_last30_mean['h_pitcher_allowed_on_base'].loc[26]
        if i == 'TEX':
            return away_last30_mean['h_pitcher_allowed_on_base'].loc[27]
        if i == 'TOR':
            return away_last30_mean['h_pitcher_allowed_on_base'].loc[28]
        if i == 'WAS':
            return away_last30_mean['h_pitcher_allowed_on_base'].loc[29]

In [45]:
def home_last_30_days_allowed_on_base(i):
        if i == 'ANA':
            return home_last30_mean['h_pitcher_allowed_on_base'].loc[0]
        if i == 'ARI':
            return home_last30_mean['h_pitcher_allowed_on_base'].loc[1]
        if i == 'ATL':
            return home_last30_mean['h_pitcher_allowed_on_base'].loc[2]
        if i == 'BAL':
            return home_last30_mean['h_pitcher_allowed_on_base'].loc[3]
        if i == 'BOS':
            return home_last30_mean['h_pitcher_allowed_on_base'].loc[4]
        if i == 'CHA':
            return home_last30_mean['h_pitcher_allowed_on_base'].loc[5]
        if i == 'CHN':
            return home_last30_mean['h_pitcher_allowed_on_base'].loc[6]
        if i == 'CIN':
            return home_last30_mean['h_pitcher_allowed_on_base'].loc[7]
        if i == 'CLE':
            return home_last30_mean['h_pitcher_allowed_on_base'].loc[8]
        if i == 'COL':
            return home_last30_mean['h_pitcher_allowed_on_base'].loc[9]
        if i == 'DET':
            return home_last30_mean['h_pitcher_allowed_on_base'].loc[10]
        if i == 'HOU':
            return home_last30_mean['h_pitcher_allowed_on_base'].loc[11]
        if i == 'KCA':
            return home_last30_mean['h_pitcher_allowed_on_base'].loc[12]
        if i == 'LAN':
            return home_last30_mean['h_pitcher_allowed_on_base'].loc[13]
        if i == 'MIA':
            return home_last30_mean['h_pitcher_allowed_on_base'].loc[14]
        if i == 'MIL':
            return home_last30_mean['h_pitcher_allowed_on_base'].loc[15]
        if i == 'MIN':
            return home_last30_mean['h_pitcher_allowed_on_base'].loc[16]
        if i == 'NYA':
            return home_last30_mean['h_pitcher_allowed_on_base'].loc[17]
        if i == 'NYN':
            return home_last30_mean['h_pitcher_allowed_on_base'].loc[18]
        if i == 'OAK':
            return home_last30_mean['h_pitcher_allowed_on_base'].loc[19]
        if i == 'PHI':
            return home_last30_mean['h_pitcher_allowed_on_base'].loc[20]
        if i == 'PIT':
            return home_last30_mean['h_pitcher_allowed_on_base'].loc[21]
        if i == 'SDN':
            return home_last30_mean['h_pitcher_allowed_on_base'].loc[22]
        if i == 'SEA':
            return home_last30_mean['h_pitcher_allowed_on_base'].loc[23]
        if i == 'SFN':
            return home_last30_mean['h_pitcher_allowed_on_base'].loc[24]
        if i == 'SLN':
            return home_last30_mean['h_pitcher_allowed_on_base'].loc[25]
        if i == 'TBA':
            return home_last30_mean['h_pitcher_allowed_on_base'].loc[26]
        if i == 'TEX':
            return home_last30_mean['h_pitcher_allowed_on_base'].loc[27]
        if i == 'TOR':
            return home_last30_mean['h_pitcher_allowed_on_base'].loc[28]
        if i == 'WAS':
            return home_last30_mean['h_pitcher_allowed_on_base'].loc[29]

In [46]:
def away_last_30_days_allowed_on_base(i):
        if i == 'ANA':
            return away_last30_mean['v_pitcher_allowed_on_base'].loc[0]
        if i == 'ARI':
            return away_last30_mean['v_pitcher_allowed_on_base'].loc[1]
        if i == 'ATL':
            return away_last30_mean['v_pitcher_allowed_on_base'].loc[2]
        if i == 'BAL':
            return away_last30_mean['v_pitcher_allowed_on_base'].loc[3]
        if i == 'BOS':
            return away_last30_mean['v_pitcher_allowed_on_base'].loc[4]
        if i == 'CHA':
            return away_last30_mean['v_pitcher_allowed_on_base'].loc[5]
        if i == 'CHN':
            return away_last30_mean['v_pitcher_allowed_on_base'].loc[6]
        if i == 'CIN':
            return away_last30_mean['v_pitcher_allowed_on_base'].loc[7]
        if i == 'CLE':
            return away_last30_mean['v_pitcher_allowed_on_base'].loc[8]
        if i == 'COL':
            return away_last30_mean['v_pitcher_allowed_on_base'].loc[9]
        if i == 'DET':
            return away_last30_mean['v_pitcher_allowed_on_base'].loc[10]
        if i == 'HOU':
            return away_last30_mean['v_pitcher_allowed_on_base'].loc[11]
        if i == 'KCA':
            return away_last30_mean['v_pitcher_allowed_on_base'].loc[12]
        if i == 'LAN':
            return away_last30_mean['v_pitcher_allowed_on_base'].loc[13]
        if i == 'MIA':
            return away_last30_mean['v_pitcher_allowed_on_base'].loc[14]
        if i == 'MIL':
            return away_last30_mean['v_pitcher_allowed_on_base'].loc[15]
        if i == 'MIN':
            return away_last30_mean['v_pitcher_allowed_on_base'].loc[16]
        if i == 'NYA':
            return away_last30_mean['v_pitcher_allowed_on_base'].loc[17]
        if i == 'NYN':
            return away_last30_mean['v_pitcher_allowed_on_base'].loc[18]
        if i == 'OAK':
            return away_last30_mean['v_pitcher_allowed_on_base'].loc[19]
        if i == 'PHI':
            return away_last30_mean['v_pitcher_allowed_on_base'].loc[20]
        if i == 'PIT':
            return away_last30_mean['v_pitcher_allowed_on_base'].loc[21]
        if i == 'SDN':
            return away_last30_mean['v_pitcher_allowed_on_base'].loc[22]
        if i == 'SEA':
            return away_last30_mean['v_pitcher_allowed_on_base'].loc[23]
        if i == 'SFN':
            return away_last30_mean['v_pitcher_allowed_on_base'].loc[24]
        if i == 'SLN':
            return away_last30_mean['v_pitcher_allowed_on_base'].loc[25]
        if i == 'TBA':
            return away_last30_mean['v_pitcher_allowed_on_base'].loc[26]
        if i == 'TEX':
            return away_last30_mean['v_pitcher_allowed_on_base'].loc[27]
        if i == 'TOR':
            return away_last30_mean['v_pitcher_allowed_on_base'].loc[28]
        if i == 'WAS':
            return away_last30_mean['v_pitcher_allowed_on_base'].loc[29]

In [47]:
def home_last_30_days_runners_advanced(i):
        if i == 'ANA':
            return home_last30_mean['h_advanced_runner'].loc[0]
        if i == 'ARI':
            return home_last30_mean['h_advanced_runner'].loc[1]
        if i == 'ATL':
            return home_last30_mean['h_advanced_runner'].loc[2]
        if i == 'BAL':
            return home_last30_mean['h_advanced_runner'].loc[3]
        if i == 'BOS':
            return home_last30_mean['h_advanced_runner'].loc[4]
        if i == 'CHA':
            return home_last30_mean['h_advanced_runner'].loc[5]
        if i == 'CHN':
            return home_last30_mean['h_advanced_runner'].loc[6]
        if i == 'CIN':
            return home_last30_mean['h_advanced_runner'].loc[7]
        if i == 'CLE':
            return home_last30_mean['h_advanced_runner'].loc[8]
        if i == 'COL':
            return home_last30_mean['h_advanced_runner'].loc[9]
        if i == 'DET':
            return home_last30_mean['h_advanced_runner'].loc[10]
        if i == 'HOU':
            return home_last30_mean['h_advanced_runner'].loc[11]
        if i == 'KCA':
            return home_last30_mean['h_advanced_runner'].loc[12]
        if i == 'LAN':
            return home_last30_mean['h_advanced_runner'].loc[13]
        if i == 'MIA':
            return home_last30_mean['h_advanced_runner'].loc[14]
        if i == 'MIL':
            return home_last30_mean['h_advanced_runner'].loc[15]
        if i == 'MIN':
            return home_last30_mean['h_advanced_runner'].loc[16]
        if i == 'NYA':
            return home_last30_mean['h_advanced_runner'].loc[17]
        if i == 'NYN':
            return home_last30_mean['h_advanced_runner'].loc[18]
        if i == 'OAK':
            return home_last30_mean['h_advanced_runner'].loc[19]
        if i == 'PHI':
            return home_last30_mean['h_advanced_runner'].loc[20]
        if i == 'PIT':
            return home_last30_mean['h_advanced_runner'].loc[21]
        if i == 'SDN':
            return home_last30_mean['h_advanced_runner'].loc[22]
        if i == 'SEA':
            return home_last30_mean['h_advanced_runner'].loc[23]
        if i == 'SFN':
            return home_last30_mean['h_advanced_runner'].loc[24]
        if i == 'SLN':
            return home_last30_mean['h_advanced_runner'].loc[25]
        if i == 'TBA':
            return home_last30_mean['h_advanced_runner'].loc[26]
        if i == 'TEX':
            return home_last30_mean['h_advanced_runner'].loc[27]
        if i == 'TOR':
            return home_last30_mean['h_advanced_runner'].loc[28]
        if i == 'WAS':
            return home_last30_mean['h_advanced_runner'].loc[29]

In [48]:
def away_last_30_days_runners_advanced(i):
        if i == 'ANA':
            return away_last30_mean['v_advanced_runner'].loc[0]
        if i == 'ARI':
            return away_last30_mean['v_advanced_runner'].loc[1]
        if i == 'ATL':
            return away_last30_mean['v_advanced_runner'].loc[2]
        if i == 'BAL':
            return away_last30_mean['v_advanced_runner'].loc[3]
        if i == 'BOS':
            return away_last30_mean['v_advanced_runner'].loc[4]
        if i == 'CHA':
            return away_last30_mean['v_advanced_runner'].loc[5]
        if i == 'CHN':
            return away_last30_mean['v_advanced_runner'].loc[6]
        if i == 'CIN':
            return away_last30_mean['v_advanced_runner'].loc[7]
        if i == 'CLE':
            return away_last30_mean['v_advanced_runner'].loc[8]
        if i == 'COL':
            return away_last30_mean['v_advanced_runner'].loc[9]
        if i == 'DET':
            return away_last30_mean['v_advanced_runner'].loc[10]
        if i == 'HOU':
            return away_last30_mean['v_advanced_runner'].loc[11]
        if i == 'KCA':
            return away_last30_mean['v_advanced_runner'].loc[12]
        if i == 'LAN':
            return away_last30_mean['v_advanced_runner'].loc[13]
        if i == 'MIA':
            return away_last30_mean['v_advanced_runner'].loc[14]
        if i == 'MIL':
            return away_last30_mean['v_advanced_runner'].loc[15]
        if i == 'MIN':
            return away_last30_mean['v_advanced_runner'].loc[16]
        if i == 'NYA':
            return away_last30_mean['v_advanced_runner'].loc[17]
        if i == 'NYN':
            return away_last30_mean['v_advanced_runner'].loc[18]
        if i == 'OAK':
            return away_last30_mean['v_advanced_runner'].loc[19]
        if i == 'PHI':
            return away_last30_mean['v_advanced_runner'].loc[20]
        if i == 'PIT':
            return away_last30_mean['v_advanced_runner'].loc[21]
        if i == 'SDN':
            return away_last30_mean['v_advanced_runner'].loc[22]
        if i == 'SEA':
            return away_last30_mean['v_advanced_runner'].loc[23]
        if i == 'SFN':
            return away_last30_mean['v_advanced_runner'].loc[24]
        if i == 'SLN':
            return away_last30_mean['v_advanced_runner'].loc[25]
        if i == 'TBA':
            return away_last30_mean['v_advanced_runner'].loc[26]
        if i == 'TEX':
            return away_last30_mean['v_advanced_runner'].loc[27]
        if i == 'TOR':
            return away_last30_mean['v_advanced_runner'].loc[28]
        if i == 'WAS':
            return away_last30_mean['v_advanced_runner'].loc[29]

In [49]:
def home_last_30_days_pitcher_allowed_runners_advanced(i):
        if i == 'ANA':
            return home_last30_mean['h_pitcher_allowed_runner_advance'].loc[0]
        if i == 'ARI':
            return home_last30_mean['h_pitcher_allowed_runner_advance'].loc[1]
        if i == 'ATL':
            return home_last30_mean['h_pitcher_allowed_runner_advance'].loc[2]
        if i == 'BAL':
            return home_last30_mean['h_pitcher_allowed_runner_advance'].loc[3]
        if i == 'BOS':
            return home_last30_mean['h_pitcher_allowed_runner_advance'].loc[4]
        if i == 'CHA':
            return home_last30_mean['h_pitcher_allowed_runner_advance'].loc[5]
        if i == 'CHN':
            return home_last30_mean['h_pitcher_allowed_runner_advance'].loc[6]
        if i == 'CIN':
            return home_last30_mean['h_pitcher_allowed_runner_advance'].loc[7]
        if i == 'CLE':
            return home_last30_mean['h_pitcher_allowed_runner_advance'].loc[8]
        if i == 'COL':
            return home_last30_mean['h_pitcher_allowed_runner_advance'].loc[9]
        if i == 'DET':
            return home_last30_mean['h_pitcher_allowed_runner_advance'].loc[10]
        if i == 'HOU':
            return home_last30_mean['h_pitcher_allowed_runner_advance'].loc[11]
        if i == 'KCA':
            return home_last30_mean['h_pitcher_allowed_runner_advance'].loc[12]
        if i == 'LAN':
            return home_last30_mean['h_pitcher_allowed_runner_advance'].loc[13]
        if i == 'MIA':
            return home_last30_mean['h_pitcher_allowed_runner_advance'].loc[14]
        if i == 'MIL':
            return home_last30_mean['h_pitcher_allowed_runner_advance'].loc[15]
        if i == 'MIN':
            return home_last30_mean['h_pitcher_allowed_runner_advance'].loc[16]
        if i == 'NYA':
            return home_last30_mean['h_pitcher_allowed_runner_advance'].loc[17]
        if i == 'NYN':
            return home_last30_mean['h_pitcher_allowed_runner_advance'].loc[18]
        if i == 'OAK':
            return home_last30_mean['h_pitcher_allowed_runner_advance'].loc[19]
        if i == 'PHI':
            return home_last30_mean['h_pitcher_allowed_runner_advance'].loc[20]
        if i == 'PIT':
            return home_last30_mean['h_pitcher_allowed_runner_advance'].loc[21]
        if i == 'SDN':
            return home_last30_mean['h_pitcher_allowed_runner_advance'].loc[22]
        if i == 'SEA':
            return home_last30_mean['h_pitcher_allowed_runner_advance'].loc[23]
        if i == 'SFN':
            return home_last30_mean['h_pitcher_allowed_runner_advance'].loc[24]
        if i == 'SLN':
            return home_last30_mean['h_pitcher_allowed_runner_advance'].loc[25]
        if i == 'TBA':
            return home_last30_mean['h_pitcher_allowed_runner_advance'].loc[26]
        if i == 'TEX':
            return home_last30_mean['h_pitcher_allowed_runner_advance'].loc[27]
        if i == 'TOR':
            return home_last30_mean['h_pitcher_allowed_runner_advance'].loc[28]
        if i == 'WAS':
            return home_last30_mean['h_pitcher_allowed_runner_advance'].loc[29]

In [50]:
def away_last_30_days_pitcher_allowed_runners_advanced(i):
        if i == 'ANA':
            return away_last30_mean['v_pitcher_allowed_runner_advance'].loc[0]
        if i == 'ARI':
            return away_last30_mean['v_pitcher_allowed_runner_advance'].loc[1]
        if i == 'ATL':
            return away_last30_mean['v_pitcher_allowed_runner_advance'].loc[2]
        if i == 'BAL':
            return away_last30_mean['v_pitcher_allowed_runner_advance'].loc[3]
        if i == 'BOS':
            return away_last30_mean['v_pitcher_allowed_runner_advance'].loc[4]
        if i == 'CHA':
            return away_last30_mean['v_pitcher_allowed_runner_advance'].loc[5]
        if i == 'CHN':
            return away_last30_mean['v_pitcher_allowed_runner_advance'].loc[6]
        if i == 'CIN':
            return away_last30_mean['v_pitcher_allowed_runner_advance'].loc[7]
        if i == 'CLE':
            return away_last30_mean['v_pitcher_allowed_runner_advance'].loc[8]
        if i == 'COL':
            return away_last30_mean['v_pitcher_allowed_runner_advance'].loc[9]
        if i == 'DET':
            return away_last30_mean['v_pitcher_allowed_runner_advance'].loc[10]
        if i == 'HOU':
            return away_last30_mean['v_pitcher_allowed_runner_advance'].loc[11]
        if i == 'KCA':
            return away_last30_mean['v_pitcher_allowed_runner_advance'].loc[12]
        if i == 'LAN':
            return away_last30_mean['v_pitcher_allowed_runner_advance'].loc[13]
        if i == 'MIA':
            return away_last30_mean['v_pitcher_allowed_runner_advance'].loc[14]
        if i == 'MIL':
            return away_last30_mean['v_pitcher_allowed_runner_advance'].loc[15]
        if i == 'MIN':
            return away_last30_mean['v_pitcher_allowed_runner_advance'].loc[16]
        if i == 'NYA':
            return away_last30_mean['v_pitcher_allowed_runner_advance'].loc[17]
        if i == 'NYN':
            return away_last30_mean['v_pitcher_allowed_runner_advance'].loc[18]
        if i == 'OAK':
            return away_last30_mean['v_pitcher_allowed_runner_advance'].loc[19]
        if i == 'PHI':
            return away_last30_mean['v_pitcher_allowed_runner_advance'].loc[20]
        if i == 'PIT':
            return away_last30_mean['v_pitcher_allowed_runner_advance'].loc[21]
        if i == 'SDN':
            return away_last30_mean['v_pitcher_allowed_runner_advance'].loc[22]
        if i == 'SEA':
            return away_last30_mean['v_pitcher_allowed_runner_advance'].loc[23]
        if i == 'SFN':
            return away_last30_mean['v_pitcher_allowed_runner_advance'].loc[24]
        if i == 'SLN':
            return away_last30_mean['v_pitcher_allowed_runner_advance'].loc[25]
        if i == 'TBA':
            return away_last30_mean['v_pitcher_allowed_runner_advance'].loc[26]
        if i == 'TEX':
            return away_last30_mean['v_pitcher_allowed_runner_advance'].loc[27]
        if i == 'TOR':
            return away_last30_mean['v_pitcher_allowed_runner_advance'].loc[28]
        if i == 'WAS':
            return away_last30_mean['v_pitcher_allowed_runner_advance'].loc[29]

In [51]:
def home_last_30_days_runners_left_on_base(i):
        if i == 'ANA':
            return home_last30_mean['h_left_on_base'].loc[0]
        if i == 'ARI':
            return home_last30_mean['h_left_on_base'].loc[1]
        if i == 'ATL':
            return home_last30_mean['h_left_on_base'].loc[2]
        if i == 'BAL':
            return home_last30_mean['h_left_on_base'].loc[3]
        if i == 'BOS':
            return home_last30_mean['h_left_on_base'].loc[4]
        if i == 'CHA':
            return home_last30_mean['h_left_on_base'].loc[5]
        if i == 'CHN':
            return home_last30_mean['h_left_on_base'].loc[6]
        if i == 'CIN':
            return home_last30_mean['h_left_on_base'].loc[7]
        if i == 'CLE':
            return home_last30_mean['h_left_on_base'].loc[8]
        if i == 'COL':
            return home_last30_mean['h_left_on_base'].loc[9]
        if i == 'DET':
            return home_last30_mean['h_left_on_base'].loc[10]
        if i == 'HOU':
            return home_last30_mean['h_left_on_base'].loc[11]
        if i == 'KCA':
            return home_last30_mean['h_left_on_base'].loc[12]
        if i == 'LAN':
            return home_last30_mean['h_left_on_base'].loc[13]
        if i == 'MIA':
            return home_last30_mean['h_left_on_base'].loc[14]
        if i == 'MIL':
            return home_last30_mean['h_left_on_base'].loc[15]
        if i == 'MIN':
            return home_last30_mean['h_left_on_base'].loc[16]
        if i == 'NYA':
            return home_last30_mean['h_left_on_base'].loc[17]
        if i == 'NYN':
            return home_last30_mean['h_left_on_base'].loc[18]
        if i == 'OAK':
            return home_last30_mean['h_left_on_base'].loc[19]
        if i == 'PHI':
            return home_last30_mean['h_left_on_base'].loc[20]
        if i == 'PIT':
            return home_last30_mean['h_left_on_base'].loc[21]
        if i == 'SDN':
            return home_last30_mean['h_left_on_base'].loc[22]
        if i == 'SEA':
            return home_last30_mean['h_left_on_base'].loc[23]
        if i == 'SFN':
            return home_last30_mean['h_left_on_base'].loc[24]
        if i == 'SLN':
            return home_last30_mean['h_left_on_base'].loc[25]
        if i == 'TBA':
            return home_last30_mean['h_left_on_base'].loc[26]
        if i == 'TEX':
            return home_last30_mean['h_left_on_base'].loc[27]
        if i == 'TOR':
            return home_last30_mean['h_left_on_base'].loc[28]
        if i == 'WAS':
            return home_last30_mean['h_left_on_base'].loc[29]

In [52]:
def away_last_30_days_runners_left_on_base(i):
        if i == 'ANA':
            return away_last30_mean['v_left_on_base'].loc[0]
        if i == 'ARI':
            return away_last30_mean['v_left_on_base'].loc[1]
        if i == 'ATL':
            return away_last30_mean['v_left_on_base'].loc[2]
        if i == 'BAL':
            return away_last30_mean['v_left_on_base'].loc[3]
        if i == 'BOS':
            return away_last30_mean['v_left_on_base'].loc[4]
        if i == 'CHA':
            return away_last30_mean['v_left_on_base'].loc[5]
        if i == 'CHN':
            return away_last30_mean['v_left_on_base'].loc[6]
        if i == 'CIN':
            return away_last30_mean['v_left_on_base'].loc[7]
        if i == 'CLE':
            return away_last30_mean['v_left_on_base'].loc[8]
        if i == 'COL':
            return away_last30_mean['v_left_on_base'].loc[9]
        if i == 'DET':
            return away_last30_mean['v_left_on_base'].loc[10]
        if i == 'HOU':
            return away_last30_mean['v_left_on_base'].loc[11]
        if i == 'KCA':
            return away_last30_mean['v_left_on_base'].loc[12]
        if i == 'LAN':
            return away_last30_mean['v_left_on_base'].loc[13]
        if i == 'MIA':
            return away_last30_mean['v_left_on_base'].loc[14]
        if i == 'MIL':
            return away_last30_mean['v_left_on_base'].loc[15]
        if i == 'MIN':
            return away_last30_mean['v_left_on_base'].loc[16]
        if i == 'NYA':
            return away_last30_mean['v_left_on_base'].loc[17]
        if i == 'NYN':
            return away_last30_mean['v_left_on_base'].loc[18]
        if i == 'OAK':
            return away_last30_mean['v_left_on_base'].loc[19]
        if i == 'PHI':
            return away_last30_mean['v_left_on_base'].loc[20]
        if i == 'PIT':
            return away_last30_mean['v_left_on_base'].loc[21]
        if i == 'SDN':
            return away_last30_mean['v_left_on_base'].loc[22]
        if i == 'SEA':
            return away_last30_mean['v_left_on_base'].loc[23]
        if i == 'SFN':
            return away_last30_mean['v_left_on_base'].loc[24]
        if i == 'SLN':
            return away_last30_mean['v_left_on_base'].loc[25]
        if i == 'TBA':
            return away_last30_mean['v_left_on_base'].loc[26]
        if i == 'TEX':
            return away_last30_mean['v_left_on_base'].loc[27]
        if i == 'TOR':
            return away_last30_mean['v_left_on_base'].loc[28]
        if i == 'WAS':
            return away_last30_mean['v_left_on_base'].loc[29]

In [53]:
def home_last_30_days_runners_stopped_from_scoring(i):
        if i == 'ANA':
            return home_last30_mean['v_left_on_base'].loc[0]
        if i == 'ARI':
            return home_last30_mean['v_left_on_base'].loc[1]
        if i == 'ATL':
            return home_last30_mean['v_left_on_base'].loc[2]
        if i == 'BAL':
            return home_last30_mean['v_left_on_base'].loc[3]
        if i == 'BOS':
            return home_last30_mean['v_left_on_base'].loc[4]
        if i == 'CHA':
            return home_last30_mean['v_left_on_base'].loc[5]
        if i == 'CHN':
            return home_last30_mean['v_left_on_base'].loc[6]
        if i == 'CIN':
            return home_last30_mean['v_left_on_base'].loc[7]
        if i == 'CLE':
            return home_last30_mean['v_left_on_base'].loc[8]
        if i == 'COL':
            return home_last30_mean['v_left_on_base'].loc[9]
        if i == 'DET':
            return home_last30_mean['v_left_on_base'].loc[10]
        if i == 'HOU':
            return home_last30_mean['v_left_on_base'].loc[11]
        if i == 'KCA':
            return home_last30_mean['v_left_on_base'].loc[12]
        if i == 'LAN':
            return home_last30_mean['v_left_on_base'].loc[13]
        if i == 'MIA':
            return home_last30_mean['v_left_on_base'].loc[14]
        if i == 'MIL':
            return home_last30_mean['v_left_on_base'].loc[15]
        if i == 'MIN':
            return home_last30_mean['v_left_on_base'].loc[16]
        if i == 'NYA':
            return home_last30_mean['v_left_on_base'].loc[17]
        if i == 'NYN':
            return home_last30_mean['v_left_on_base'].loc[18]
        if i == 'OAK':
            return home_last30_mean['v_left_on_base'].loc[19]
        if i == 'PHI':
            return home_last30_mean['v_left_on_base'].loc[20]
        if i == 'PIT':
            return home_last30_mean['v_left_on_base'].loc[21]
        if i == 'SDN':
            return home_last30_mean['v_left_on_base'].loc[22]
        if i == 'SEA':
            return home_last30_mean['v_left_on_base'].loc[23]
        if i == 'SFN':
            return home_last30_mean['v_left_on_base'].loc[24]
        if i == 'SLN':
            return home_last30_mean['v_left_on_base'].loc[25]
        if i == 'TBA':
            return home_last30_mean['v_left_on_base'].loc[26]
        if i == 'TEX':
            return home_last30_mean['v_left_on_base'].loc[27]
        if i == 'TOR':
            return home_last30_mean['v_left_on_base'].loc[28]
        if i == 'WAS':
            return home_last30_mean['v_left_on_base'].loc[29]

In [54]:
def away_last_30_days_runners_stopped_from_scoring(i):
        if i == 'ANA':
            return away_last30_mean['h_left_on_base'].loc[0]
        if i == 'ARI':
            return away_last30_mean['h_left_on_base'].loc[1]
        if i == 'ATL':
            return away_last30_mean['h_left_on_base'].loc[2]
        if i == 'BAL':
            return away_last30_mean['h_left_on_base'].loc[3]
        if i == 'BOS':
            return away_last30_mean['h_left_on_base'].loc[4]
        if i == 'CHA':
            return away_last30_mean['h_left_on_base'].loc[5]
        if i == 'CHN':
            return away_last30_mean['h_left_on_base'].loc[6]
        if i == 'CIN':
            return away_last30_mean['h_left_on_base'].loc[7]
        if i == 'CLE':
            return away_last30_mean['h_left_on_base'].loc[8]
        if i == 'COL':
            return away_last30_mean['h_left_on_base'].loc[9]
        if i == 'DET':
            return away_last30_mean['h_left_on_base'].loc[10]
        if i == 'HOU':
            return away_last30_mean['h_left_on_base'].loc[11]
        if i == 'KCA':
            return away_last30_mean['h_left_on_base'].loc[12]
        if i == 'LAN':
            return away_last30_mean['h_left_on_base'].loc[13]
        if i == 'MIA':
            return away_last30_mean['h_left_on_base'].loc[14]
        if i == 'MIL':
            return away_last30_mean['h_left_on_base'].loc[15]
        if i == 'MIN':
            return away_last30_mean['h_left_on_base'].loc[16]
        if i == 'NYA':
            return away_last30_mean['h_left_on_base'].loc[17]
        if i == 'NYN':
            return away_last30_mean['h_left_on_base'].loc[18]
        if i == 'OAK':
            return away_last30_mean['h_left_on_base'].loc[19]
        if i == 'PHI':
            return away_last30_mean['h_left_on_base'].loc[20]
        if i == 'PIT':
            return away_last30_mean['h_left_on_base'].loc[21]
        if i == 'SDN':
            return away_last30_mean['h_left_on_base'].loc[22]
        if i == 'SEA':
            return away_last30_mean['h_left_on_base'].loc[23]
        if i == 'SFN':
            return away_last30_mean['h_left_on_base'].loc[24]
        if i == 'SLN':
            return away_last30_mean['h_left_on_base'].loc[25]
        if i == 'TBA':
            return away_last30_mean['h_left_on_base'].loc[26]
        if i == 'TEX':
            return away_last30_mean['h_left_on_base'].loc[27]
        if i == 'TOR':
            return away_last30_mean['h_left_on_base'].loc[28]
        if i == 'WAS':
            return away_last30_mean['h_left_on_base'].loc[29]

In [55]:
def home_last_30_days_attendance(i):
        if i == 'ANA':
            return home_last30_mean['attendance'].loc[0]
        if i == 'ARI':
            return home_last30_mean['attendance'].loc[1]
        if i == 'ATL':
            return home_last30_mean['attendance'].loc[2]
        if i == 'BAL':
            return home_last30_mean['attendance'].loc[3]
        if i == 'BOS':
            return home_last30_mean['attendance'].loc[4]
        if i == 'CHA':
            return home_last30_mean['attendance'].loc[5]
        if i == 'CHN':
            return home_last30_mean['attendance'].loc[6]
        if i == 'CIN':
            return home_last30_mean['attendance'].loc[7]
        if i == 'CLE':
            return home_last30_mean['attendance'].loc[8]
        if i == 'COL':
            return home_last30_mean['attendance'].loc[9]
        if i == 'DET':
            return home_last30_mean['attendance'].loc[10]
        if i == 'HOU':
            return home_last30_mean['attendance'].loc[11]
        if i == 'KCA':
            return home_last30_mean['attendance'].loc[12]
        if i == 'LAN':
            return home_last30_mean['attendance'].loc[13]
        if i == 'MIA':
            return home_last30_mean['attendance'].loc[14]
        if i == 'MIL':
            return home_last30_mean['attendance'].loc[15]
        if i == 'MIN':
            return home_last30_mean['attendance'].loc[16]
        if i == 'NYA':
            return home_last30_mean['attendance'].loc[17]
        if i == 'NYN':
            return home_last30_mean['attendance'].loc[18]
        if i == 'OAK':
            return home_last30_mean['attendance'].loc[19]
        if i == 'PHI':
            return home_last30_mean['attendance'].loc[20]
        if i == 'PIT':
            return home_last30_mean['attendance'].loc[21]
        if i == 'SDN':
            return home_last30_mean['attendance'].loc[22]
        if i == 'SEA':
            return home_last30_mean['attendance'].loc[23]
        if i == 'SFN':
            return home_last30_mean['attendance'].loc[24]
        if i == 'SLN':
            return home_last30_mean['attendance'].loc[25]
        if i == 'TBA':
            return home_last30_mean['attendance'].loc[26]
        if i == 'TEX':
            return home_last30_mean['attendance'].loc[27]
        if i == 'TOR':
            return home_last30_mean['attendance'].loc[28]
        if i == 'WAS':
            return home_last30_mean['attendance'].loc[29]

In [56]:
def away_last_30_days_attendance(i):
        if i == 'ANA':
            return away_last30_mean['attendance'].loc[0]
        if i == 'ARI':
            return away_last30_mean['attendance'].loc[1]
        if i == 'ATL':
            return away_last30_mean['attendance'].loc[2]
        if i == 'BAL':
            return away_last30_mean['attendance'].loc[3]
        if i == 'BOS':
            return away_last30_mean['attendance'].loc[4]
        if i == 'CHA':
            return away_last30_mean['attendance'].loc[5]
        if i == 'CHN':
            return away_last30_mean['attendance'].loc[6]
        if i == 'CIN':
            return away_last30_mean['attendance'].loc[7]
        if i == 'CLE':
            return away_last30_mean['attendance'].loc[8]
        if i == 'COL':
            return away_last30_mean['attendance'].loc[9]
        if i == 'DET':
            return away_last30_mean['attendance'].loc[10]
        if i == 'HOU':
            return away_last30_mean['attendance'].loc[11]
        if i == 'KCA':
            return away_last30_mean['attendance'].loc[12]
        if i == 'LAN':
            return away_last30_mean['attendance'].loc[13]
        if i == 'MIA':
            return away_last30_mean['attendance'].loc[14]
        if i == 'MIL':
            return away_last30_mean['attendance'].loc[15]
        if i == 'MIN':
            return away_last30_mean['attendance'].loc[16]
        if i == 'NYA':
            return away_last30_mean['attendance'].loc[17]
        if i == 'NYN':
            return away_last30_mean['attendance'].loc[18]
        if i == 'OAK':
            return away_last30_mean['attendance'].loc[19]
        if i == 'PHI':
            return away_last30_mean['attendance'].loc[20]
        if i == 'PIT':
            return away_last30_mean['attendance'].loc[21]
        if i == 'SDN':
            return away_last30_mean['attendance'].loc[22]
        if i == 'SEA':
            return away_last30_mean['attendance'].loc[23]
        if i == 'SFN':
            return away_last30_mean['attendance'].loc[24]
        if i == 'SLN':
            return away_last30_mean['attendance'].loc[25]
        if i == 'TBA':
            return away_last30_mean['attendance'].loc[26]
        if i == 'TEX':
            return away_last30_mean['attendance'].loc[27]
        if i == 'TOR':
            return away_last30_mean['attendance'].loc[28]
        if i == 'WAS':
            return away_last30_mean['attendance'].loc[29]

In [57]:
def home_last_30_days_being_struck_out(i):
        if i == 'ANA':
            return home_last30_mean['h_strikeouts'].loc[0]
        if i == 'ARI':
            return home_last30_mean['h_strikeouts'].loc[1]
        if i == 'ATL':
            return home_last30_mean['h_strikeouts'].loc[2]
        if i == 'BAL':
            return home_last30_mean['h_strikeouts'].loc[3]
        if i == 'BOS':
            return home_last30_mean['h_strikeouts'].loc[4]
        if i == 'CHA':
            return home_last30_mean['h_strikeouts'].loc[5]
        if i == 'CHN':
            return home_last30_mean['h_strikeouts'].loc[6]
        if i == 'CIN':
            return home_last30_mean['h_strikeouts'].loc[7]
        if i == 'CLE':
            return home_last30_mean['h_strikeouts'].loc[8]
        if i == 'COL':
            return home_last30_mean['h_strikeouts'].loc[9]
        if i == 'DET':
            return home_last30_mean['h_strikeouts'].loc[10]
        if i == 'HOU':
            return home_last30_mean['h_strikeouts'].loc[11]
        if i == 'KCA':
            return home_last30_mean['h_strikeouts'].loc[12]
        if i == 'LAN':
            return home_last30_mean['h_strikeouts'].loc[13]
        if i == 'MIA':
            return home_last30_mean['h_strikeouts'].loc[14]
        if i == 'MIL':
            return home_last30_mean['h_strikeouts'].loc[15]
        if i == 'MIN':
            return home_last30_mean['h_strikeouts'].loc[16]
        if i == 'NYA':
            return home_last30_mean['h_strikeouts'].loc[17]
        if i == 'NYN':
            return home_last30_mean['h_strikeouts'].loc[18]
        if i == 'OAK':
            return home_last30_mean['h_strikeouts'].loc[19]
        if i == 'PHI':
            return home_last30_mean['h_strikeouts'].loc[20]
        if i == 'PIT':
            return home_last30_mean['h_strikeouts'].loc[21]
        if i == 'SDN':
            return home_last30_mean['h_strikeouts'].loc[22]
        if i == 'SEA':
            return home_last30_mean['h_strikeouts'].loc[23]
        if i == 'SFN':
            return home_last30_mean['h_strikeouts'].loc[24]
        if i == 'SLN':
            return home_last30_mean['h_strikeouts'].loc[25]
        if i == 'TBA':
            return home_last30_mean['h_strikeouts'].loc[26]
        if i == 'TEX':
            return home_last30_mean['h_strikeouts'].loc[27]
        if i == 'TOR':
            return home_last30_mean['h_strikeouts'].loc[28]
        if i == 'WAS':
            return home_last30_mean['h_strikeouts'].loc[29]

In [58]:
def away_last_30_days_being_struck_out(i):
        if i == 'ANA':
            return away_last30_mean['v_strikeouts'].loc[0]
        if i == 'ARI':
            return away_last30_mean['v_strikeouts'].loc[1]
        if i == 'ATL':
            return away_last30_mean['v_strikeouts'].loc[2]
        if i == 'BAL':
            return away_last30_mean['v_strikeouts'].loc[3]
        if i == 'BOS':
            return away_last30_mean['v_strikeouts'].loc[4]
        if i == 'CHA':
            return away_last30_mean['v_strikeouts'].loc[5]
        if i == 'CHN':
            return away_last30_mean['v_strikeouts'].loc[6]
        if i == 'CIN':
            return away_last30_mean['v_strikeouts'].loc[7]
        if i == 'CLE':
            return away_last30_mean['v_strikeouts'].loc[8]
        if i == 'COL':
            return away_last30_mean['v_strikeouts'].loc[9]
        if i == 'DET':
            return away_last30_mean['v_strikeouts'].loc[10]
        if i == 'HOU':
            return away_last30_mean['v_strikeouts'].loc[11]
        if i == 'KCA':
            return away_last30_mean['v_strikeouts'].loc[12]
        if i == 'LAN':
            return away_last30_mean['v_strikeouts'].loc[13]
        if i == 'MIA':
            return away_last30_mean['v_strikeouts'].loc[14]
        if i == 'MIL':
            return away_last30_mean['v_strikeouts'].loc[15]
        if i == 'MIN':
            return away_last30_mean['v_strikeouts'].loc[16]
        if i == 'NYA':
            return away_last30_mean['v_strikeouts'].loc[17]
        if i == 'NYN':
            return away_last30_mean['v_strikeouts'].loc[18]
        if i == 'OAK':
            return away_last30_mean['v_strikeouts'].loc[19]
        if i == 'PHI':
            return away_last30_mean['v_strikeouts'].loc[20]
        if i == 'PIT':
            return away_last30_mean['v_strikeouts'].loc[21]
        if i == 'SDN':
            return away_last30_mean['v_strikeouts'].loc[22]
        if i == 'SEA':
            return away_last30_mean['v_strikeouts'].loc[23]
        if i == 'SFN':
            return away_last30_mean['v_strikeouts'].loc[24]
        if i == 'SLN':
            return away_last30_mean['v_strikeouts'].loc[25]
        if i == 'TBA':
            return away_last30_mean['v_strikeouts'].loc[26]
        if i == 'TEX':
            return away_last30_mean['v_strikeouts'].loc[27]
        if i == 'TOR':
            return away_last30_mean['v_strikeouts'].loc[28]
        if i == 'WAS':
            return away_last30_mean['v_strikeouts'].loc[29]

In [59]:
def home_last_30_days_striking_out_opponents(i):
        if i == 'ANA':
            return home_last30_mean['v_strikeouts'].loc[0]
        if i == 'ARI':
            return home_last30_mean['v_strikeouts'].loc[1]
        if i == 'ATL':
            return home_last30_mean['v_strikeouts'].loc[2]
        if i == 'BAL':
            return home_last30_mean['v_strikeouts'].loc[3]
        if i == 'BOS':
            return home_last30_mean['v_strikeouts'].loc[4]
        if i == 'CHA':
            return home_last30_mean['v_strikeouts'].loc[5]
        if i == 'CHN':
            return home_last30_mean['v_strikeouts'].loc[6]
        if i == 'CIN':
            return home_last30_mean['v_strikeouts'].loc[7]
        if i == 'CLE':
            return home_last30_mean['v_strikeouts'].loc[8]
        if i == 'COL':
            return home_last30_mean['v_strikeouts'].loc[9]
        if i == 'DET':
            return home_last30_mean['v_strikeouts'].loc[10]
        if i == 'HOU':
            return home_last30_mean['v_strikeouts'].loc[11]
        if i == 'KCA':
            return home_last30_mean['v_strikeouts'].loc[12]
        if i == 'LAN':
            return home_last30_mean['v_strikeouts'].loc[13]
        if i == 'MIA':
            return home_last30_mean['v_strikeouts'].loc[14]
        if i == 'MIL':
            return home_last30_mean['v_strikeouts'].loc[15]
        if i == 'MIN':
            return home_last30_mean['v_strikeouts'].loc[16]
        if i == 'NYA':
            return home_last30_mean['v_strikeouts'].loc[17]
        if i == 'NYN':
            return home_last30_mean['v_strikeouts'].loc[18]
        if i == 'OAK':
            return home_last30_mean['v_strikeouts'].loc[19]
        if i == 'PHI':
            return home_last30_mean['v_strikeouts'].loc[20]
        if i == 'PIT':
            return home_last30_mean['v_strikeouts'].loc[21]
        if i == 'SDN':
            return home_last30_mean['v_strikeouts'].loc[22]
        if i == 'SEA':
            return home_last30_mean['v_strikeouts'].loc[23]
        if i == 'SFN':
            return home_last30_mean['v_strikeouts'].loc[24]
        if i == 'SLN':
            return home_last30_mean['v_strikeouts'].loc[25]
        if i == 'TBA':
            return home_last30_mean['v_strikeouts'].loc[26]
        if i == 'TEX':
            return home_last30_mean['v_strikeouts'].loc[27]
        if i == 'TOR':
            return home_last30_mean['v_strikeouts'].loc[28]
        if i == 'WAS':
            return home_last30_mean['v_strikeouts'].loc[29]

In [60]:
def away_last_30_days_striking_out_opponents(i):
        if i == 'ANA':
            return away_last30_mean['h_strikeouts'].loc[0]
        if i == 'ARI':
            return away_last30_mean['h_strikeouts'].loc[1]
        if i == 'ATL':
            return away_last30_mean['h_strikeouts'].loc[2]
        if i == 'BAL':
            return away_last30_mean['h_strikeouts'].loc[3]
        if i == 'BOS':
            return away_last30_mean['h_strikeouts'].loc[4]
        if i == 'CHA':
            return away_last30_mean['h_strikeouts'].loc[5]
        if i == 'CHN':
            return away_last30_mean['h_strikeouts'].loc[6]
        if i == 'CIN':
            return away_last30_mean['h_strikeouts'].loc[7]
        if i == 'CLE':
            return away_last30_mean['h_strikeouts'].loc[8]
        if i == 'COL':
            return away_last30_mean['h_strikeouts'].loc[9]
        if i == 'DET':
            return away_last30_mean['h_strikeouts'].loc[10]
        if i == 'HOU':
            return away_last30_mean['h_strikeouts'].loc[11]
        if i == 'KCA':
            return away_last30_mean['h_strikeouts'].loc[12]
        if i == 'LAN':
            return away_last30_mean['h_strikeouts'].loc[13]
        if i == 'MIA':
            return away_last30_mean['h_strikeouts'].loc[14]
        if i == 'MIL':
            return away_last30_mean['h_strikeouts'].loc[15]
        if i == 'MIN':
            return away_last30_mean['h_strikeouts'].loc[16]
        if i == 'NYA':
            return away_last30_mean['h_strikeouts'].loc[17]
        if i == 'NYN':
            return away_last30_mean['h_strikeouts'].loc[18]
        if i == 'OAK':
            return away_last30_mean['h_strikeouts'].loc[19]
        if i == 'PHI':
            return away_last30_mean['h_strikeouts'].loc[20]
        if i == 'PIT':
            return away_last30_mean['h_strikeouts'].loc[21]
        if i == 'SDN':
            return away_last30_mean['h_strikeouts'].loc[22]
        if i == 'SEA':
            return away_last30_mean['h_strikeouts'].loc[23]
        if i == 'SFN':
            return away_last30_mean['h_strikeouts'].loc[24]
        if i == 'SLN':
            return away_last30_mean['h_strikeouts'].loc[25]
        if i == 'TBA':
            return away_last30_mean['h_strikeouts'].loc[26]
        if i == 'TEX':
            return away_last30_mean['h_strikeouts'].loc[27]
        if i == 'TOR':
            return away_last30_mean['h_strikeouts'].loc[28]
        if i == 'WAS':
            return away_last30_mean['h_strikeouts'].loc[29]

In [61]:
def home_last_30_days_errors_made(i):
        if i == 'ANA':
            return home_last30_mean['h_errors'].loc[0]
        if i == 'ARI':
            return home_last30_mean['h_errors'].loc[1]
        if i == 'ATL':
            return home_last30_mean['h_errors'].loc[2]
        if i == 'BAL':
            return home_last30_mean['h_errors'].loc[3]
        if i == 'BOS':
            return home_last30_mean['h_errors'].loc[4]
        if i == 'CHA':
            return home_last30_mean['h_errors'].loc[5]
        if i == 'CHN':
            return home_last30_mean['h_errors'].loc[6]
        if i == 'CIN':
            return home_last30_mean['h_errors'].loc[7]
        if i == 'CLE':
            return home_last30_mean['h_errors'].loc[8]
        if i == 'COL':
            return home_last30_mean['h_errors'].loc[9]
        if i == 'DET':
            return home_last30_mean['h_errors'].loc[10]
        if i == 'HOU':
            return home_last30_mean['h_errors'].loc[11]
        if i == 'KCA':
            return home_last30_mean['h_errors'].loc[12]
        if i == 'LAN':
            return home_last30_mean['h_errors'].loc[13]
        if i == 'MIA':
            return home_last30_mean['h_errors'].loc[14]
        if i == 'MIL':
            return home_last30_mean['h_errors'].loc[15]
        if i == 'MIN':
            return home_last30_mean['h_errors'].loc[16]
        if i == 'NYA':
            return home_last30_mean['h_errors'].loc[17]
        if i == 'NYN':
            return home_last30_mean['h_errors'].loc[18]
        if i == 'OAK':
            return home_last30_mean['h_errors'].loc[19]
        if i == 'PHI':
            return home_last30_mean['h_errors'].loc[20]
        if i == 'PIT':
            return home_last30_mean['h_errors'].loc[21]
        if i == 'SDN':
            return home_last30_mean['h_errors'].loc[22]
        if i == 'SEA':
            return home_last30_mean['h_errors'].loc[23]
        if i == 'SFN':
            return home_last30_mean['h_errors'].loc[24]
        if i == 'SLN':
            return home_last30_mean['h_errors'].loc[25]
        if i == 'TBA':
            return home_last30_mean['h_errors'].loc[26]
        if i == 'TEX':
            return home_last30_mean['h_errors'].loc[27]
        if i == 'TOR':
            return home_last30_mean['h_errors'].loc[28]
        if i == 'WAS':
            return home_last30_mean['h_errors'].loc[29]

In [62]:
def away_last_30_days_errors_made(i):
        if i == 'ANA':
            return away_last30_mean['v_errors'].loc[0]
        if i == 'ARI':
            return away_last30_mean['v_errors'].loc[1]
        if i == 'ATL':
            return away_last30_mean['v_errors'].loc[2]
        if i == 'BAL':
            return away_last30_mean['v_errors'].loc[3]
        if i == 'BOS':
            return away_last30_mean['v_errors'].loc[4]
        if i == 'CHA':
            return away_last30_mean['v_errors'].loc[5]
        if i == 'CHN':
            return away_last30_mean['v_errors'].loc[6]
        if i == 'CIN':
            return away_last30_mean['v_errors'].loc[7]
        if i == 'CLE':
            return away_last30_mean['v_errors'].loc[8]
        if i == 'COL':
            return away_last30_mean['v_errors'].loc[9]
        if i == 'DET':
            return away_last30_mean['v_errors'].loc[10]
        if i == 'HOU':
            return away_last30_mean['v_errors'].loc[11]
        if i == 'KCA':
            return away_last30_mean['v_errors'].loc[12]
        if i == 'LAN':
            return away_last30_mean['v_errors'].loc[13]
        if i == 'MIA':
            return away_last30_mean['v_errors'].loc[14]
        if i == 'MIL':
            return away_last30_mean['v_errors'].loc[15]
        if i == 'MIN':
            return away_last30_mean['v_errors'].loc[16]
        if i == 'NYA':
            return away_last30_mean['v_errors'].loc[17]
        if i == 'NYN':
            return away_last30_mean['v_errors'].loc[18]
        if i == 'OAK':
            return away_last30_mean['v_errors'].loc[19]
        if i == 'PHI':
            return away_last30_mean['v_errors'].loc[20]
        if i == 'PIT':
            return away_last30_mean['v_errors'].loc[21]
        if i == 'SDN':
            return away_last30_mean['v_errors'].loc[22]
        if i == 'SEA':
            return away_last30_mean['v_errors'].loc[23]
        if i == 'SFN':
            return away_last30_mean['v_errors'].loc[24]
        if i == 'SLN':
            return away_last30_mean['v_errors'].loc[25]
        if i == 'TBA':
            return away_last30_mean['v_errors'].loc[26]
        if i == 'TEX':
            return away_last30_mean['v_errors'].loc[27]
        if i == 'TOR':
            return away_last30_mean['v_errors'].loc[28]
        if i == 'WAS':
            return away_last30_mean['v_errors'].loc[29]

In [63]:
def home_last_30_days_double_plays_made(i):
        if i == 'ANA':
            return home_last30_mean['h_double_plays'].loc[0]
        if i == 'ARI':
            return home_last30_mean['h_double_plays'].loc[1]
        if i == 'ATL':
            return home_last30_mean['h_double_plays'].loc[2]
        if i == 'BAL':
            return home_last30_mean['h_double_plays'].loc[3]
        if i == 'BOS':
            return home_last30_mean['h_double_plays'].loc[4]
        if i == 'CHA':
            return home_last30_mean['h_double_plays'].loc[5]
        if i == 'CHN':
            return home_last30_mean['h_double_plays'].loc[6]
        if i == 'CIN':
            return home_last30_mean['h_double_plays'].loc[7]
        if i == 'CLE':
            return home_last30_mean['h_double_plays'].loc[8]
        if i == 'COL':
            return home_last30_mean['h_double_plays'].loc[9]
        if i == 'DET':
            return home_last30_mean['h_double_plays'].loc[10]
        if i == 'HOU':
            return home_last30_mean['h_double_plays'].loc[11]
        if i == 'KCA':
            return home_last30_mean['h_double_plays'].loc[12]
        if i == 'LAN':
            return home_last30_mean['h_double_plays'].loc[13]
        if i == 'MIA':
            return home_last30_mean['h_double_plays'].loc[14]
        if i == 'MIL':
            return home_last30_mean['h_double_plays'].loc[15]
        if i == 'MIN':
            return home_last30_mean['h_double_plays'].loc[16]
        if i == 'NYA':
            return home_last30_mean['h_double_plays'].loc[17]
        if i == 'NYN':
            return home_last30_mean['h_double_plays'].loc[18]
        if i == 'OAK':
            return home_last30_mean['h_double_plays'].loc[19]
        if i == 'PHI':
            return home_last30_mean['h_double_plays'].loc[20]
        if i == 'PIT':
            return home_last30_mean['h_double_plays'].loc[21]
        if i == 'SDN':
            return home_last30_mean['h_double_plays'].loc[22]
        if i == 'SEA':
            return home_last30_mean['h_double_plays'].loc[23]
        if i == 'SFN':
            return home_last30_mean['h_double_plays'].loc[24]
        if i == 'SLN':
            return home_last30_mean['h_double_plays'].loc[25]
        if i == 'TBA':
            return home_last30_mean['h_double_plays'].loc[26]
        if i == 'TEX':
            return home_last30_mean['h_double_plays'].loc[27]
        if i == 'TOR':
            return home_last30_mean['h_double_plays'].loc[28]
        if i == 'WAS':
            return home_last30_mean['h_double_plays'].loc[29]

In [64]:
def away_last_30_days_double_plays_made(i):
        if i == 'ANA':
            return away_last30_mean['v_double_plays'].loc[0]
        if i == 'ARI':
            return away_last30_mean['v_double_plays'].loc[1]
        if i == 'ATL':
            return away_last30_mean['v_double_plays'].loc[2]
        if i == 'BAL':
            return away_last30_mean['v_double_plays'].loc[3]
        if i == 'BOS':
            return away_last30_mean['v_double_plays'].loc[4]
        if i == 'CHA':
            return away_last30_mean['v_double_plays'].loc[5]
        if i == 'CHN':
            return away_last30_mean['v_double_plays'].loc[6]
        if i == 'CIN':
            return away_last30_mean['v_double_plays'].loc[7]
        if i == 'CLE':
            return away_last30_mean['v_double_plays'].loc[8]
        if i == 'COL':
            return away_last30_mean['v_double_plays'].loc[9]
        if i == 'DET':
            return away_last30_mean['v_double_plays'].loc[10]
        if i == 'HOU':
            return away_last30_mean['v_double_plays'].loc[11]
        if i == 'KCA':
            return away_last30_mean['v_double_plays'].loc[12]
        if i == 'LAN':
            return away_last30_mean['v_double_plays'].loc[13]
        if i == 'MIA':
            return away_last30_mean['v_double_plays'].loc[14]
        if i == 'MIL':
            return away_last30_mean['v_double_plays'].loc[15]
        if i == 'MIN':
            return away_last30_mean['v_double_plays'].loc[16]
        if i == 'NYA':
            return away_last30_mean['v_double_plays'].loc[17]
        if i == 'NYN':
            return away_last30_mean['v_double_plays'].loc[18]
        if i == 'OAK':
            return away_last30_mean['v_double_plays'].loc[19]
        if i == 'PHI':
            return away_last30_mean['v_double_plays'].loc[20]
        if i == 'PIT':
            return away_last30_mean['v_double_plays'].loc[21]
        if i == 'SDN':
            return away_last30_mean['v_double_plays'].loc[22]
        if i == 'SEA':
            return away_last30_mean['v_double_plays'].loc[23]
        if i == 'SFN':
            return away_last30_mean['v_double_plays'].loc[24]
        if i == 'SLN':
            return away_last30_mean['v_double_plays'].loc[25]
        if i == 'TBA':
            return away_last30_mean['v_double_plays'].loc[26]
        if i == 'TEX':
            return away_last30_mean['v_double_plays'].loc[27]
        if i == 'TOR':
            return away_last30_mean['v_double_plays'].loc[28]
        if i == 'WAS':
            return away_last30_mean['v_double_plays'].loc[29]

In [65]:
def home_last_30_days_double_plays_hit_into(i):
        if i == 'ANA':
            return home_last30_mean['v_double_plays'].loc[0]
        if i == 'ARI':
            return home_last30_mean['v_double_plays'].loc[1]
        if i == 'ATL':
            return home_last30_mean['v_double_plays'].loc[2]
        if i == 'BAL':
            return home_last30_mean['v_double_plays'].loc[3]
        if i == 'BOS':
            return home_last30_mean['v_double_plays'].loc[4]
        if i == 'CHA':
            return home_last30_mean['v_double_plays'].loc[5]
        if i == 'CHN':
            return home_last30_mean['v_double_plays'].loc[6]
        if i == 'CIN':
            return home_last30_mean['v_double_plays'].loc[7]
        if i == 'CLE':
            return home_last30_mean['v_double_plays'].loc[8]
        if i == 'COL':
            return home_last30_mean['v_double_plays'].loc[9]
        if i == 'DET':
            return home_last30_mean['v_double_plays'].loc[10]
        if i == 'HOU':
            return home_last30_mean['v_double_plays'].loc[11]
        if i == 'KCA':
            return home_last30_mean['v_double_plays'].loc[12]
        if i == 'LAN':
            return home_last30_mean['v_double_plays'].loc[13]
        if i == 'MIA':
            return home_last30_mean['v_double_plays'].loc[14]
        if i == 'MIL':
            return home_last30_mean['v_double_plays'].loc[15]
        if i == 'MIN':
            return home_last30_mean['v_double_plays'].loc[16]
        if i == 'NYA':
            return home_last30_mean['v_double_plays'].loc[17]
        if i == 'NYN':
            return home_last30_mean['v_double_plays'].loc[18]
        if i == 'OAK':
            return home_last30_mean['v_double_plays'].loc[19]
        if i == 'PHI':
            return home_last30_mean['v_double_plays'].loc[20]
        if i == 'PIT':
            return home_last30_mean['v_double_plays'].loc[21]
        if i == 'SDN':
            return home_last30_mean['v_double_plays'].loc[22]
        if i == 'SEA':
            return home_last30_mean['v_double_plays'].loc[23]
        if i == 'SFN':
            return home_last30_mean['v_double_plays'].loc[24]
        if i == 'SLN':
            return home_last30_mean['v_double_plays'].loc[25]
        if i == 'TBA':
            return home_last30_mean['v_double_plays'].loc[26]
        if i == 'TEX':
            return home_last30_mean['v_double_plays'].loc[27]
        if i == 'TOR':
            return home_last30_mean['v_double_plays'].loc[28]
        if i == 'WAS':
            return home_last30_mean['v_double_plays'].loc[29]

In [66]:
def away_last_30_days_double_plays_hit_into(i):
        if i == 'ANA':
            return away_last30_mean['h_double_plays'].loc[0]
        if i == 'ARI':
            return away_last30_mean['h_double_plays'].loc[1]
        if i == 'ATL':
            return away_last30_mean['h_double_plays'].loc[2]
        if i == 'BAL':
            return away_last30_mean['h_double_plays'].loc[3]
        if i == 'BOS':
            return away_last30_mean['h_double_plays'].loc[4]
        if i == 'CHA':
            return away_last30_mean['h_double_plays'].loc[5]
        if i == 'CHN':
            return away_last30_mean['h_double_plays'].loc[6]
        if i == 'CIN':
            return away_last30_mean['h_double_plays'].loc[7]
        if i == 'CLE':
            return away_last30_mean['h_double_plays'].loc[8]
        if i == 'COL':
            return away_last30_mean['h_double_plays'].loc[9]
        if i == 'DET':
            return away_last30_mean['h_double_plays'].loc[10]
        if i == 'HOU':
            return away_last30_mean['h_double_plays'].loc[11]
        if i == 'KCA':
            return away_last30_mean['h_double_plays'].loc[12]
        if i == 'LAN':
            return away_last30_mean['h_double_plays'].loc[13]
        if i == 'MIA':
            return away_last30_mean['h_double_plays'].loc[14]
        if i == 'MIL':
            return away_last30_mean['h_double_plays'].loc[15]
        if i == 'MIN':
            return away_last30_mean['h_double_plays'].loc[16]
        if i == 'NYA':
            return away_last30_mean['h_double_plays'].loc[17]
        if i == 'NYN':
            return away_last30_mean['h_double_plays'].loc[18]
        if i == 'OAK':
            return away_last30_mean['h_double_plays'].loc[19]
        if i == 'PHI':
            return away_last30_mean['h_double_plays'].loc[20]
        if i == 'PIT':
            return away_last30_mean['h_double_plays'].loc[21]
        if i == 'SDN':
            return away_last30_mean['h_double_plays'].loc[22]
        if i == 'SEA':
            return away_last30_mean['h_double_plays'].loc[23]
        if i == 'SFN':
            return away_last30_mean['h_double_plays'].loc[24]
        if i == 'SLN':
            return away_last30_mean['h_double_plays'].loc[25]
        if i == 'TBA':
            return away_last30_mean['h_double_plays'].loc[26]
        if i == 'TEX':
            return away_last30_mean['h_double_plays'].loc[27]
        if i == 'TOR':
            return away_last30_mean['h_double_plays'].loc[28]
        if i == 'WAS':
            return away_last30_mean['h_double_plays'].loc[29]

In [67]:
def home_last_30_days_runs_scored(i):
        if i == 'ANA':
            return home_last30_mean['h_score'].loc[0]
        if i == 'ARI':
            return home_last30_mean['h_score'].loc[1]
        if i == 'ATL':
            return home_last30_mean['h_score'].loc[2]
        if i == 'BAL':
            return home_last30_mean['h_score'].loc[3]
        if i == 'BOS':
            return home_last30_mean['h_score'].loc[4]
        if i == 'CHA':
            return home_last30_mean['h_score'].loc[5]
        if i == 'CHN':
            return home_last30_mean['h_score'].loc[6]
        if i == 'CIN':
            return home_last30_mean['h_score'].loc[7]
        if i == 'CLE':
            return home_last30_mean['h_score'].loc[8]
        if i == 'COL':
            return home_last30_mean['h_score'].loc[9]
        if i == 'DET':
            return home_last30_mean['h_score'].loc[10]
        if i == 'HOU':
            return home_last30_mean['h_score'].loc[11]
        if i == 'KCA':
            return home_last30_mean['h_score'].loc[12]
        if i == 'LAN':
            return home_last30_mean['h_score'].loc[13]
        if i == 'MIA':
            return home_last30_mean['h_score'].loc[14]
        if i == 'MIL':
            return home_last30_mean['h_score'].loc[15]
        if i == 'MIN':
            return home_last30_mean['h_score'].loc[16]
        if i == 'NYA':
            return home_last30_mean['h_score'].loc[17]
        if i == 'NYN':
            return home_last30_mean['h_score'].loc[18]
        if i == 'OAK':
            return home_last30_mean['h_score'].loc[19]
        if i == 'PHI':
            return home_last30_mean['h_score'].loc[20]
        if i == 'PIT':
            return home_last30_mean['h_score'].loc[21]
        if i == 'SDN':
            return home_last30_mean['h_score'].loc[22]
        if i == 'SEA':
            return home_last30_mean['h_score'].loc[23]
        if i == 'SFN':
            return home_last30_mean['h_score'].loc[24]
        if i == 'SLN':
            return home_last30_mean['h_score'].loc[25]
        if i == 'TBA':
            return home_last30_mean['h_score'].loc[26]
        if i == 'TEX':
            return home_last30_mean['h_score'].loc[27]
        if i == 'TOR':
            return home_last30_mean['h_score'].loc[28]
        if i == 'WAS':
            return home_last30_mean['h_score'].loc[29]

In [68]:
def away_last_30_days_runs_scored(i):
        if i == 'ANA':
            return away_last30_mean['v_score'].loc[0]
        if i == 'ARI':
            return away_last30_mean['v_score'].loc[1]
        if i == 'ATL':
            return away_last30_mean['v_score'].loc[2]
        if i == 'BAL':
            return away_last30_mean['v_score'].loc[3]
        if i == 'BOS':
            return away_last30_mean['v_score'].loc[4]
        if i == 'CHA':
            return away_last30_mean['v_score'].loc[5]
        if i == 'CHN':
            return away_last30_mean['v_score'].loc[6]
        if i == 'CIN':
            return away_last30_mean['v_score'].loc[7]
        if i == 'CLE':
            return away_last30_mean['v_score'].loc[8]
        if i == 'COL':
            return away_last30_mean['v_score'].loc[9]
        if i == 'DET':
            return away_last30_mean['v_score'].loc[10]
        if i == 'HOU':
            return away_last30_mean['v_score'].loc[11]
        if i == 'KCA':
            return away_last30_mean['v_score'].loc[12]
        if i == 'LAN':
            return away_last30_mean['v_score'].loc[13]
        if i == 'MIA':
            return away_last30_mean['v_score'].loc[14]
        if i == 'MIL':
            return away_last30_mean['v_score'].loc[15]
        if i == 'MIN':
            return away_last30_mean['v_score'].loc[16]
        if i == 'NYA':
            return away_last30_mean['v_score'].loc[17]
        if i == 'NYN':
            return away_last30_mean['v_score'].loc[18]
        if i == 'OAK':
            return away_last30_mean['v_score'].loc[19]
        if i == 'PHI':
            return away_last30_mean['v_score'].loc[20]
        if i == 'PIT':
            return away_last30_mean['v_score'].loc[21]
        if i == 'SDN':
            return away_last30_mean['v_score'].loc[22]
        if i == 'SEA':
            return away_last30_mean['v_score'].loc[23]
        if i == 'SFN':
            return away_last30_mean['v_score'].loc[24]
        if i == 'SLN':
            return away_last30_mean['v_score'].loc[25]
        if i == 'TBA':
            return away_last30_mean['v_score'].loc[26]
        if i == 'TEX':
            return away_last30_mean['v_score'].loc[27]
        if i == 'TOR':
            return away_last30_mean['v_score'].loc[28]
        if i == 'WAS':
            return away_last30_mean['v_score'].loc[29]

In [69]:
def home_last_30_days_runs_allowed(i):
        if i == 'ANA':
            return home_last30_mean['v_score'].loc[0]
        if i == 'ARI':
            return home_last30_mean['v_score'].loc[1]
        if i == 'ATL':
            return home_last30_mean['v_score'].loc[2]
        if i == 'BAL':
            return home_last30_mean['v_score'].loc[3]
        if i == 'BOS':
            return home_last30_mean['v_score'].loc[4]
        if i == 'CHA':
            return home_last30_mean['v_score'].loc[5]
        if i == 'CHN':
            return home_last30_mean['v_score'].loc[6]
        if i == 'CIN':
            return home_last30_mean['v_score'].loc[7]
        if i == 'CLE':
            return home_last30_mean['v_score'].loc[8]
        if i == 'COL':
            return home_last30_mean['v_score'].loc[9]
        if i == 'DET':
            return home_last30_mean['v_score'].loc[10]
        if i == 'HOU':
            return home_last30_mean['v_score'].loc[11]
        if i == 'KCA':
            return home_last30_mean['v_score'].loc[12]
        if i == 'LAN':
            return home_last30_mean['v_score'].loc[13]
        if i == 'MIA':
            return home_last30_mean['v_score'].loc[14]
        if i == 'MIL':
            return home_last30_mean['v_score'].loc[15]
        if i == 'MIN':
            return home_last30_mean['v_score'].loc[16]
        if i == 'NYA':
            return home_last30_mean['v_score'].loc[17]
        if i == 'NYN':
            return home_last30_mean['v_score'].loc[18]
        if i == 'OAK':
            return home_last30_mean['v_score'].loc[19]
        if i == 'PHI':
            return home_last30_mean['v_score'].loc[20]
        if i == 'PIT':
            return home_last30_mean['v_score'].loc[21]
        if i == 'SDN':
            return home_last30_mean['v_score'].loc[22]
        if i == 'SEA':
            return home_last30_mean['v_score'].loc[23]
        if i == 'SFN':
            return home_last30_mean['v_score'].loc[24]
        if i == 'SLN':
            return home_last30_mean['v_score'].loc[25]
        if i == 'TBA':
            return home_last30_mean['v_score'].loc[26]
        if i == 'TEX':
            return home_last30_mean['v_score'].loc[27]
        if i == 'TOR':
            return home_last30_mean['v_score'].loc[28]
        if i == 'WAS':
            return home_last30_mean['v_score'].loc[29]

In [70]:
def away_last_30_days_runs_allowed(i):
        if i == 'ANA':
            return away_last30_mean['h_score'].loc[0]
        if i == 'ARI':
            return away_last30_mean['h_score'].loc[1]
        if i == 'ATL':
            return away_last30_mean['h_score'].loc[2]
        if i == 'BAL':
            return away_last30_mean['h_score'].loc[3]
        if i == 'BOS':
            return away_last30_mean['h_score'].loc[4]
        if i == 'CHA':
            return away_last30_mean['h_score'].loc[5]
        if i == 'CHN':
            return away_last30_mean['h_score'].loc[6]
        if i == 'CIN':
            return away_last30_mean['h_score'].loc[7]
        if i == 'CLE':
            return away_last30_mean['h_score'].loc[8]
        if i == 'COL':
            return away_last30_mean['h_score'].loc[9]
        if i == 'DET':
            return away_last30_mean['h_score'].loc[10]
        if i == 'HOU':
            return away_last30_mean['h_score'].loc[11]
        if i == 'KCA':
            return away_last30_mean['h_score'].loc[12]
        if i == 'LAN':
            return away_last30_mean['h_score'].loc[13]
        if i == 'MIA':
            return away_last30_mean['h_score'].loc[14]
        if i == 'MIL':
            return away_last30_mean['h_score'].loc[15]
        if i == 'MIN':
            return away_last30_mean['h_score'].loc[16]
        if i == 'NYA':
            return away_last30_mean['h_score'].loc[17]
        if i == 'NYN':
            return away_last30_mean['h_score'].loc[18]
        if i == 'OAK':
            return away_last30_mean['h_score'].loc[19]
        if i == 'PHI':
            return away_last30_mean['h_score'].loc[20]
        if i == 'PIT':
            return away_last30_mean['h_score'].loc[21]
        if i == 'SDN':
            return away_last30_mean['h_score'].loc[22]
        if i == 'SEA':
            return away_last30_mean['h_score'].loc[23]
        if i == 'SFN':
            return away_last30_mean['h_score'].loc[24]
        if i == 'SLN':
            return away_last30_mean['h_score'].loc[25]
        if i == 'TBA':
            return away_last30_mean['h_score'].loc[26]
        if i == 'TEX':
            return away_last30_mean['h_score'].loc[27]
        if i == 'TOR':
            return away_last30_mean['h_score'].loc[28]
        if i == 'WAS':
            return away_last30_mean['h_score'].loc[29]

# Applying Functions

In [71]:
df['h_2016_avg_rd'] = df['h_name'].apply(home_start_szn_rd)

In [72]:
df['v_2016_avg_rd'] = df['v_name'].apply(away_start_szn_rd)

In [73]:
df['h_2016_win_rate'] = df['h_name'].apply(home_start_szn_W)

In [74]:
df['v_2016_win_rate'] = df['v_name'].apply(away_start_szn_W)

In [75]:
df['h_2016_avg_hits'] = df['h_name'].apply(home_start_szn_hits)

In [76]:
df['v_2016_avg_hits'] = df['v_name'].apply(away_start_szn_hits)

In [77]:
df['h_2016_avg_hits_allowed'] = df['h_name'].apply(home_start_szn_hits_allowed)

In [78]:
df['v_2016_avg_hits_allowed'] = df['v_name'].apply(away_start_szn_hits_allowed)

In [79]:
df['h_2016_avg_xbh'] = df['h_name'].apply(home_start_szn_xbh)

In [80]:
df['v_2016_avg_xbh'] = df['v_name'].apply(away_start_szn_xbh)

In [81]:
df['h_2016_avg_xbh_allowed'] = df['h_name'].apply(home_last_30_days_xbh_allowed)

In [82]:
df['v_2016_avg_xbh_allowed'] = df['v_name'].apply(away_last_30_days_xbh_allowed)

In [83]:
df['h_2016_avg_on_base_by_pitcher'] = df['h_name'].apply(home_last_30_days_on_base_by_pitcher)

In [84]:
df['v_2016_avg_on_base_by_pitcher'] = df['v_name'].apply(away_last_30_days_on_base_by_pitcher)

In [85]:
df['h_2016_avg_runners_advanced'] = df['h_name'].apply(home_last_30_days_runners_advanced)

In [86]:
df['v_2016_avg_runners_advanced'] = df['v_name'].apply(away_last_30_days_runners_advanced)

In [87]:
df['h_2016_avg_pitcher_allowed_runners_advance'] = df['h_name'].apply(home_last_30_days_pitcher_allowed_runners_advanced)

In [88]:
df['v_2016_avg_pitcher_allowed_runners_advance'] = df['v_name'].apply(away_last_30_days_pitcher_allowed_runners_advanced)

In [89]:
df['h_2016_avg_runners_left_on_base'] = df['h_name'].apply(home_last_30_days_runners_left_on_base)

In [90]:
df['v_2016_avg_runners_left_on_base'] = df['v_name'].apply(away_last_30_days_runners_left_on_base)

In [91]:
df['h_2016_avg_runners_stopped_from_scoring'] = df['h_name'].apply(home_last_30_days_runners_stopped_from_scoring)

In [92]:
df['v_2016_avg_runners_stopped_from_scoring'] = df['v_name'].apply(away_last_30_days_runners_stopped_from_scoring)

In [93]:
df['h_2016_avg_allowed_on_base'] = df['h_name'].apply(home_last_30_days_allowed_on_base)

In [94]:
df['v_2016_avg_allowed_on_base'] = df['v_name'].apply(away_last_30_days_allowed_on_base)

In [95]:
df['h_2016_avg_attendance'] = df['h_name'].apply(home_last_30_days_attendance)

In [96]:
df['v_2016_avg_attendance'] = df['v_name'].apply(away_last_30_days_attendance)

In [97]:
df['h_2016_avg_getting_struk_out'] = df['h_name'].apply(home_last_30_days_being_struck_out)

In [98]:
df['v_2016_avg_getting_struck_out'] = df['v_name'].apply(away_last_30_days_being_struck_out)

In [99]:
df['h_2016_avg_striking_out_opponents'] = df['h_name'].apply(home_last_30_days_striking_out_opponents)

In [100]:
df['v_2016_avg_striking_out_opponents'] = df['v_name'].apply(away_last_30_days_striking_out_opponents)

In [101]:
df['h_2016_avg_errors_made'] = df['h_name'].apply(home_last_30_days_errors_made)

In [102]:
df['v_2016_avg_errors_made'] = df['v_name'].apply(away_last_30_days_errors_made)

In [103]:
df['h_2016_avg_double_plays_made'] = df['h_name'].apply(home_last_30_days_double_plays_made)

In [104]:
df['v_2016_avg_double_plays_made'] = df['v_name'].apply(away_last_30_days_double_plays_made)

In [105]:
df['h_2016_avg_double_plays_hit_into'] = df['h_name'].apply(home_last_30_days_double_plays_hit_into)

In [106]:
df['v_2016_avg_double_plays_hit_into'] = df['v_name'].apply(away_last_30_days_double_plays_hit_into)

In [107]:
df['h_2016_avg_runs_scored'] = df['h_name'].apply(home_last_30_days_runs_scored)

In [108]:
df['v_2016_avg_runs_scored'] = df['v_name'].apply(away_last_30_days_runs_scored)

In [109]:
df['h_2016_avg_runs_allowed'] = df['h_name'].apply(home_last_30_days_runs_allowed)

In [110]:
df['v_2016_avg_runs_allowed'] = df['v_name'].apply(away_last_30_days_runs_allowed)

In [113]:
df.to_csv('df3', index = False)